In [3]:
import pandas as pd 
import numpy as np
import datetime
import importlib
from datetime import datetime,timedelta,date
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from pulp import LpVariable, LpProblem, LpMinimize, lpSum, LpStatus ,LpMaximize,PULP_CBC_CMD

import space_opt_util as space_util
importlib.reload(space_util)

<module 'space_opt_util' from 'C:\\Users\\40103735\\OneDrive - Anheuser-Busch InBev\\Sejuti\\MAZ\\Space Optimization\\Honduras\\Modeling\\space_opt_util.py'>

In [4]:
fecha = '14-09-2023'
fecha_dt = datetime.strptime(fecha, '%d-%m-%Y')
##############
#fecha_dt = date.today()
####################

fecha_str=datetime.strftime(fecha_dt, '%d-%m-%Y')
fecha_new_str= datetime.strftime(fecha_dt, "%d.%m.%Y")
day = fecha_dt.strftime("%A") 

#### what is this block saying?? what if the day is saturday? - Sejuti
### each day is named here
if day=='Sunday':
    day_of_week=day    
elif day!='Saturday':
    day_of_week="Weekday"
else:
    day_of_week=day

In [5]:
default_start_time ='00:00:01'

In [6]:
fecha_str

'14-09-2023'

In [7]:
day

'Thursday'

### SKU Attribute file

In [8]:
sku_description=pd.read_excel('Data/HO Master Data Materials FG 2023.xlsx')

In [9]:
sku_description.head(n=2)

,Sap Code,Description,Size,Pack,Category,Brand,Box x Hls,Box x Pallet,Weight x Box,Units x Box,Active Code,Weight x Pallet,HLS x Box
0,H1004,BARENA EXPORTACION 12OZ 24UD,12 ONZ,VIDRIO NR,CERVEZA,BARENA,11.741,49.0,39.06,24,No,1981.94,0.085172
1,H1005,PORT ROYAL EXPORTACION 12OZ,12 ONZ,VIDRIO NR,CERVEZA,PORT ROYAL,11.741,49.0,39.06,24,No,1981.94,0.085172


In [10]:
def read_sku_description(sku_description):
    #reads sku description 
    sku_description.columns = sku_description.columns.str.strip()
    #print(sku_description.columns)
    #remove rows where code is null and hl per pallet is null
    sku_description=sku_description[~sku_description['Sap Code'].isnull()]
    sku_description=sku_description[~sku_description['Weight x Pallet'].isnull()]
    sku_description=sku_description[~sku_description['Box x Pallet'].isnull()]
    sku_description=sku_description.drop_duplicates(subset=['Sap Code'],keep='first')
    sku_description=sku_description.rename(columns={'Sap Code': 'codigo','Weight x Pallet':'pound_weight_per_pallet','Box x Pallet':'package_per_pallet'})
    ### Adding another 54 pounds - weight of SKU carrier?
    sku_description['pound_weight_per_pallet']=sku_description['pound_weight_per_pallet']+54
    sku_description['kg_weight_per_pallet']=sku_description['pound_weight_per_pallet']*0.453592
    return sku_description

In [11]:
sku_description=read_sku_description(sku_description)

In [12]:
sku_description.head(n=2)

,codigo,Description,Size,Pack,Category,Brand,Box x Hls,package_per_pallet,Weight x Box,Units x Box,Active Code,pound_weight_per_pallet,HLS x Box,kg_weight_per_pallet
0,H1004,BARENA EXPORTACION 12OZ 24UD,12 ONZ,VIDRIO NR,CERVEZA,BARENA,11.741,49.0,39.06,24,No,2035.94,0.085172,923.486096
1,H1005,PORT ROYAL EXPORTACION 12OZ,12 ONZ,VIDRIO NR,CERVEZA,PORT ROYAL,11.741,49.0,39.06,24,No,2035.94,0.085172,923.486096


### Fleet data- Attribute and Currently availability and Ficha level Restriction

Fleet Attribute

In [13]:
def get_static_fleet_attribute_data(df_fleet):
    #Read and preprocess static fleet attribute data 
    df_fleet.columns = df_fleet.columns.str.strip()
    df_fleet=df_fleet.rename(columns={'Transporte2': 'Tipo de Vehículo','Empresa':'Modelo','Remolque2':'Ficha','Pallets':'Capacidad Pallets'})
    df_fleet['Capacidad Ton']=np.where(df_fleet['Tipo de Vehículo']=='Terceros',37,41)
    df_fleet['Capacidad KG_max'] =df_fleet['Capacidad Ton']*1000
    #df_fleet['Capacidad Ton'] =df_fleet['Peso Lbs']*0.00110231
    df_fleet_list=df_fleet[~df_fleet['Tipo de Vehículo'].isnull()]
    df_fleet_list['BREWERY']='all'
    #df_fleet_list['Capacidad Ton']= np.where(df_fleet_list['Capacidad Pallets']==24.0, 40.0, df_fleet_list['Capacidad Ton'])
    df_fleet_list['Horas de utilidad'] = '24 Hr'
    df_fleet_list=df_fleet_list[['Ficha', 'Tipo de Vehículo', 'Modelo','Capacidad KG_max','Capacidad Ton', 'Capacidad Pallets','Horas de utilidad','BREWERY']]
    #df_fleet_list=df_fleet_list[~(df_fleet_list.Ficha.isin(ficha_do_not_use_list)) ]
    return df_fleet_list

In [14]:
df_fleet=pd.read_excel(r'Data/Fleet.xlsx',sheet_name='Remolque')

In [15]:
df_fleet_list=get_static_fleet_attribute_data(df_fleet)

In [16]:
df_fleet_list.head(n=2)

,Ficha,Tipo de Vehículo,Modelo,Capacidad KG_max,Capacidad Ton,Capacidad Pallets,Horas de utilidad,BREWERY
0,HN-MR88,Hibrida,NUÑEZ,41000,41,28,24 Hr,all
1,HN-MR93,Hibrida,EMTRATECA,41000,41,28,24 Hr,all


In [17]:
df_fleet_list[df_fleet_list['Ficha']=='HN-DR01']

,Ficha,Tipo de Vehículo,Modelo,Capacidad KG_max,Capacidad Ton,Capacidad Pallets,Horas de utilidad,BREWERY
229,HN-DR01,Terceros,TOM Del Cid,37000,37,24,24 Hr,all


Current Fleet data

In [18]:
def format2(str1):
    ##convert time from 1900-01-01 02:00:00	 to 02:00:00
    result=str1
    if len(str1.split())==2:
        result=str1.split()[1]
    return result 

In [19]:
def format1(str1):
    ##convert time from 1900-01-01 02:00:00	 to 02:00:00
    result=str1
    if len(str1.split())==2:
        result=str1.split()[0]
    return result 

In [20]:
def assign_initial_time(current_day_fleet_siders,fecha_next_str,default_start_time,day):
    import datetime
    from datetime import timedelta
    Fleet_start_date_time_org=pd.to_datetime(fecha_next_str+" "+default_start_time, format='%d-%m-%Y %H:%M:%S')
    if day=='Monday':
        Max_reach_time_at_destiny=Fleet_start_date_time_org+datetime.timedelta(hours=17)
    else:
        Max_reach_time_at_destiny=Fleet_start_date_time_org+datetime.timedelta(hours=23)
    current_day_fleet_siders['Fleet_start_time']=default_start_time
    current_day_fleet_siders['Fleet_start_date_time']=pd.to_datetime(fecha_next_str+" "+default_start_time, format='%d-%m-%Y %H:%M:%S')
    current_day_fleet_siders['Fleet_start_date_time_org']=Fleet_start_date_time_org
    current_day_fleet_siders['Max_reach_time_at_destiny']=Max_reach_time_at_destiny
    #print(type(Fleet_start_date_time_org))
    #print(type(Max_reach_time_at_destiny))
    current_day_fleet_siders['Fleet_start_time']=pd.to_datetime(current_day_fleet_siders['Fleet_start_time'], format='%H:%M:%S').dt.time
    return current_day_fleet_siders

In [21]:
def get_current_date_available_sider(df_fleet_list,Fleet_availability,Fleet_weight_tractor,Fleet_weight_remolque,default_start_time,fecha_next_str,day): 
    Fleet_availability=Fleet_availability[['Combinations Final']]
    Fleet_availability=Fleet_availability.dropna(subset = ['Combinations Final'])
    Fleet_availability['Remolque'] =Fleet_availability['Combinations Final'].astype(str).apply(format2)
    Fleet_availability['Tractor'] =Fleet_availability['Combinations Final'].astype(str).apply(format1)
    Fleet_availability=Fleet_availability.merge(Fleet_weight_tractor,left_on='Tractor',right_on='Tractor ID ',how='left').merge(Fleet_weight_remolque[['Remolque_ID', 'Weight_of_Remolque_in_KG']],left_on='Remolque',right_on='Remolque_ID',how='left')
    Fleet_availability=Fleet_availability.merge(df_fleet_list, left_on='Remolque',right_on='Ficha',how='left')
    Fleet_availability['Capacidad KG']=Fleet_availability['Capacidad KG_max']-(Fleet_availability['Weight of tractor in KG ']+Fleet_availability['Weight_of_Remolque_in_KG'])
    Fleet_availability = Fleet_availability.dropna(how='any',axis=0) 
    #current_day_fleet=current_day_fleet[~(current_day_fleet.Ficha.isin(ficha_do_not_use_list)) ]
    #current_day_fleet=df_fleet_list[df_fleet_list['Ficha'].isin(Current_fleet_list)]
    #lets add code to the brewery name 
#     dict1={'CND':'DO31','HN':'HN01'}
#     current_day_fleet['BREWERY_CODE']=current_day_fleet['BREWERY'].map(dict1)
    current_day_fleet_siders=Fleet_availability.copy(deep=True)
    current_day_fleet_siders['Tipo'] =current_day_fleet_siders['Tipo de Vehículo']
    #current_day_fleet_siders=current_day_fleet[current_day_fleet['Tipo de Vehículo']=='COLA P.']
    #print(default_start_time)
    current_day_fleet_siders=assign_initial_time(current_day_fleet_siders,fecha_next_str,default_start_time,day)
    current_day_fleet_siders['Ficha']=Fleet_availability['Combinations Final']
    current_day_fleet_siders=current_day_fleet_siders[['Ficha', 'Remolque','Tipo de Vehículo', 'Modelo', 'Capacidad KG', 'Capacidad Ton',
       'Capacidad Pallets', 'Horas de utilidad', 'BREWERY', 'Tipo',
       'Fleet_start_time', 'Fleet_start_date_time',
       'Fleet_start_date_time_org', 'Max_reach_time_at_destiny']]
    #current_day_fleet_siders_CND=current_day_fleet_siders[current_day_fleet_siders.BREWERY=='CND']
    #current_day_fleet_siders_HN=current_day_fleet_siders[current_day_fleet_siders.BREWERY=='HN']
    return current_day_fleet_siders


In [22]:
# Fleet_availability =pd.read_excel(r'Data/Combinations Trucks.xlsx',skiprows=1,sheet_name='Hoja1')
# Fleet_weight_tractor=pd.read_excel(r'Data/Combinations Trucks.xlsx',sheet_name='Tractor Info')
# Fleet_weight_remolque=pd.read_excel(r'Data/Combinations Trucks.xlsx',sheet_name='Remolque Info')
# Fleet_availability=Fleet_availability[['Combinations Final']]
# Fleet_availability=Fleet_availability.dropna(subset = ['Combinations Final'])
# Fleet_availability['Remolque'] =Fleet_availability['Combinations Final'].astype(str).apply(format2)
# Fleet_availability['Tractor'] =Fleet_availability['Combinations Final'].astype(str).apply(format1)
# Fleet_availability=Fleet_availability.merge(Fleet_weight_tractor,left_on='Tractor',right_on='Tractor ID ',how='left').merge(Fleet_weight_remolque[['Remolque_ID', 'Weight_of_Remolque_in_KG']],left_on='Remolque',right_on='Remolque_ID',how='left')
# Fleet_availability=Fleet_availability.merge(df_fleet_list, left_on='Remolque',right_on='Ficha',how='left')
# Fleet_availability['Capacidad KG']=Fleet_availability['Capacidad KG_max']-(Fleet_availability['Weight of tractor in KG ']+Fleet_availability['Weight_of_Remolque_in_KG'])
# Fleet_availability = Fleet_availability.dropna(how='any',axis=0) 

In [23]:
Fleet_availability =pd.read_excel(r'Data/Combinations Trucks.xlsx',skiprows=1,sheet_name='Hoja1')
Fleet_weight_tractor=pd.read_excel(r'Data/Combinations Trucks.xlsx',sheet_name='Tractor Info')
Fleet_weight_remolque=pd.read_excel(r'Data/Combinations Trucks.xlsx',sheet_name='Remolque Info')

In [24]:
current_day_fleet_siders= get_current_date_available_sider(df_fleet_list,Fleet_availability,Fleet_weight_tractor,Fleet_weight_remolque,default_start_time,fecha_str,day).reset_index().drop(['index'],axis=1)

In [25]:
current_day_fleet_siders[current_day_fleet_siders['Ficha']=='HN-DC01  HN-DR01']

,Ficha,Remolque,Tipo de Vehículo,Modelo,Capacidad KG,Capacidad Ton,Capacidad Pallets,Horas de utilidad,BREWERY,Tipo,Fleet_start_time,Fleet_start_date_time,Fleet_start_date_time_org,Max_reach_time_at_destiny


In [26]:
current_day_fleet_siders[current_day_fleet_siders['Ficha']=='HN-TC42  HN-TR11']

,Ficha,Remolque,Tipo de Vehículo,Modelo,Capacidad KG,Capacidad Ton,Capacidad Pallets,Horas de utilidad,BREWERY,Tipo,Fleet_start_time,Fleet_start_date_time,Fleet_start_date_time_org,Max_reach_time_at_destiny
158,HN-TC42 HN-TR11,HN-TR11,Terceros,TOM,20008.425128,37,24,24 Hr,all,Terceros,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01


### Restrictions Brewery-DC

In [27]:
def Restrictions_input(current_day_fleet_siders,Restrictions):
    Restrictions=Restrictions1[['Remolque','Restriccion.1','Origen','SKU']]
    Restrictions=Restrictions.dropna(subset=['Remolque'])
    Restrictions['Remolque']='HN-'+Restrictions['Remolque'].astype(str)
    ficha_do_not_use_list=list(Restrictions[Restrictions['Restriccion.1'].isin(['Accidentado','Decomisado'])]['Remolque'].unique())
    current_day_fleet_siders=current_day_fleet_siders[~(current_day_fleet_siders.Ficha.isin(ficha_do_not_use_list))]
    ##### Merge with Fleet data to get final restrictions
    current_day_fleet_siders_Rest=current_day_fleet_siders.merge(Restrictions, left_on='Remolque',right_on='Remolque',how='left').drop(['Remolque'],axis=1).rename(columns={'Restriccion.1':'Ficha_Restriction_destiny','Origen':'Ficha_Restriction_origin','SKU':'Ficha_Restriction_sku'})
    current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny'].isin(['Monitor','Ninguna']),'No restriction',np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=='Dollie','DH00',np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=='saba','DH11',
    np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=='Talanga individual','DH10-I',np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=='Choluteca individual','DH03-I',
    np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=='Juticalpa individual','DH15-I',np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=='La Ceiba individual','DH02',
    np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=='La Granja','DH01',np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=='Choluteca','DH03',current_day_fleet_siders_Rest['Ficha_Restriction_destiny'])))))))))
                                                                        
    current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=current_day_fleet_siders_Rest['Ficha_Restriction_destiny'].fillna('No restriction')
    #dict_d={'Talanga individual':'DH10-I','Choluteca individual':'DH03-I','Juticalpa individual':'DH15-I','La Ceiba individual':'DH02','La Granja':'DH01','Saba':'DH11','San Pedro Sula':'DH00','Choluteca':'DH03'}
    #current_day_fleet_siders_Rest['Ficha_Restriction_destiny_code']=current_day_fleet_siders_Rest['Ficha_Restriction_destiny'].map(dict_d)
    current_day_fleet_siders_Rest['Ficha_Restriction_origin']=current_day_fleet_siders_Rest['Ficha_Restriction_origin'].fillna('No restriction')
    current_day_fleet_siders_Rest['Ficha_Restriction_sku']=current_day_fleet_siders_Rest['Ficha_Restriction_sku'].fillna('No restriction')
    return current_day_fleet_siders_Rest

In [28]:
Restrictions1 =pd.read_excel('Data/Restricciones de Flota.xlsx',sheet_name='Restricciones',skiprows=2)
current_day_fleet_siders=Restrictions_input(current_day_fleet_siders,Restrictions1)
print(current_day_fleet_siders.shape)
current_day_fleet_siders.head(n=1)

(312, 16)


,Ficha,Tipo de Vehículo,Modelo,Capacidad KG,Capacidad Ton,Capacidad Pallets,Horas de utilidad,BREWERY,Tipo,Fleet_start_time,Fleet_start_date_time,Fleet_start_date_time_org,Max_reach_time_at_destiny,Ficha_Restriction_destiny,Ficha_Restriction_origin,Ficha_Restriction_sku
0,HN-1333 HN-1064,Propia,Loginhsa,21504.576752,41,24,24 Hr,all,Propia,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,DH00,No restriction,No restriction


In [29]:
current_day_fleet_siders[current_day_fleet_siders['Ficha_Restriction_destiny']!='No restriction']['Ficha'].shape[0]

142

In [30]:
current_day_fleet_siders[current_day_fleet_siders['Ficha_Restriction_destiny']=='AO']

,Ficha,Tipo de Vehículo,Modelo,Capacidad KG,Capacidad Ton,Capacidad Pallets,Horas de utilidad,BREWERY,Tipo,Fleet_start_time,Fleet_start_date_time,Fleet_start_date_time_org,Max_reach_time_at_destiny,Ficha_Restriction_destiny,Ficha_Restriction_origin,Ficha_Restriction_sku
159,HN-TC42 HN-TR11,Terceros,TOM,20008.425128,37,24,24 Hr,all,Terceros,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,AO,No restriction,No restriction
160,HN-TC56 HN-TR02,Terceros,TOM,19618.319710,37,24,24 Hr,all,Terceros,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,AO,No restriction,No restriction


### Restrictions Brewery-Customers

### ATTENTION AND DISTANCE DETAILS 

In [31]:
def format1(str1):
    ##convert time from 1900-01-01 02:00:00	 to 02:00:00
    result=str1
    if len(str1.split(" "))==2:
        result=str1.split(" ")[1]
    return result 

#### Origin

In [32]:
origin_attention_df1 =pd.read_excel('Data/MASTER DATA VENTANA DE ATENCION T2.xlsx',sheet_name='Origen',skiprows=1)

In [33]:
origin_attention_df1.columns

Index(['Turno', 'Inicio', 'Fin', 'T1', 'Unnamed: 4', 'Turno.1', 'Inicio.1',
       'Fin.1', 'T1.1', 'Unnamed: 9', 'Turno.2', 'Inicio.2', 'Fin.2', 'T1.2',
       'Unnamed: 14', 'Turno.3', 'Inicio.3', 'Fin.3', 'T1.3', 'Unnamed: 19',
       'Turno.4', 'Inicio.4', 'Fin.4', 'T1.4'],
      dtype='object')

In [34]:
origin_attention_df1=origin_attention_df1.drop(['Unnamed: 4','Unnamed: 9','Unnamed: 14','Unnamed: 19'],axis=1)
origin_attention_df1['COD']='SH01'
origin_attention_df1['COD.1']='DH22'
origin_attention_df1['COD.2']='DH20'
origin_attention_df1['COD.3']='BH01'
origin_attention_df1['COD.4']='DH19'

origin_dict={'SH01':'CSD','DH22':'KEYMART','DH20':'POLVORIN','BH01':'BEER','DH19':'ALDESA'}
origin_dict

origin_attention_df=origin_attention_df1[['Inicio','Fin','T1','COD']].append(origin_attention_df1[['Inicio.1','Fin.1','T1.1','COD.1']].rename(columns={'Inicio.1':'Inicio','Fin.1':'Fin','T1.1':'T1','COD.1':'COD'})).append(origin_attention_df1[['Inicio.2','Fin.2','T1.2','COD.2']].rename(columns={'Inicio.2':'Inicio','Fin.2':'Fin','T1.2':'T1','COD.2':'COD'})).append(origin_attention_df1[['Inicio.3','Fin.3','T1.3','COD.3']].rename(columns={'Inicio.3':'Inicio','Fin.3':'Fin','T1.3':'T1','COD.3':'COD'})).append(origin_attention_df1[['Inicio.4','Fin.4','T1.4','COD.4']].rename(columns={'Inicio.4':'Inicio','Fin.4':'Fin','T1.4':'T1','COD.4':'COD'}))
origin_attention_df['CDD']=origin_attention_df['COD'].map(origin_dict)

In [35]:
origin_attention_df=origin_attention_df.rename(columns={'Inicio':'Attention_Start','Fin':'Attention_End','T1':'Weekday_attention'})
origin_attention_df=origin_attention_df.dropna(subset=['Attention_Start'])
origin_attention_df=origin_attention_df.fillna(0)
origin_attention_df=origin_attention_df.reset_index()

In [36]:
origin_attention_df['Attention_Start']=origin_attention_df['Attention_Start'].astype(str).apply(format1)
origin_attention_df['Attention_End']=origin_attention_df['Attention_End'].astype(str).apply(format1)  

origin_attention_df['Attention_Start'] = pd.to_datetime(origin_attention_df['Attention_Start'], format='%H:%M:%S').dt.time
origin_attention_df['Attention_End'] = pd.to_datetime(origin_attention_df['Attention_End'], format='%H:%M:%S').dt.time

origin_attention_df['Saturday_attention']=0
origin_attention_df['Sunday_attention']=0

origin_attention_df1=origin_attention_df[['COD','CDD','Attention_Start','Attention_End','Weekday_attention','Saturday_attention','Sunday_attention']]

Defining a few variable, which will be in loop later

In [37]:
# origin = 'BH01'
# destiny = 'DH15'
# #Fleet_start_date_time ='2023-09-14 06:00:01'
## return - whether trip can start or not
## if trip can start when can it start - it can start at 8

#### Destination - Brewery - DC

In [38]:
destiny_attention_df1_weekday =pd.read_excel('Data/MASTER DATA VENTANA DE ATENCION T2.xlsx',sheet_name='Ventanas T1',skiprows=3)
destiny_attention_df1_weekday=destiny_attention_df1_weekday.head(n=24)

destiny_attention_df1_Sat =pd.read_excel('Data/MASTER DATA VENTANA DE ATENCION T2.xlsx',sheet_name='Ventanas T1',skiprows=33)
destiny_attention_df1_Sat=destiny_attention_df1_Sat.head(n=24)

destiny_attention_df1_Sun =pd.read_excel('Data/MASTER DATA VENTANA DE ATENCION T2.xlsx',sheet_name='Ventanas T1',skiprows=63)
destiny_attention_df1_Sun=destiny_attention_df1_Sun.head(n=24)

In [39]:
destiny_attention_df1_weekday=destiny_attention_df1_weekday.drop(['Unnamed: 4','Unnamed: 9','Unnamed: 14','Unnamed: 19','Unnamed: 24',
                        'Unnamed: 29','Unnamed: 34','Unnamed: 39','Unnamed: 44','Unnamed: 49','Unnamed: 54','Unnamed: 59','Unnamed: 64','Unnamed: 69'],axis=1)
destiny_attention_df1_weekday['COD']='DH01'
destiny_attention_df1_weekday['COD.1']='DH09'
destiny_attention_df1_weekday['COD.2']='DH03'
destiny_attention_df1_weekday['COD.3']='DH15'
destiny_attention_df1_weekday['COD.4']='DH10'
destiny_attention_df1_weekday['COD.5']='DH08'
destiny_attention_df1_weekday['COD.6']='DH13'
destiny_attention_df1_weekday['COD.7']='DH05'
destiny_attention_df1_weekday['COD.8']='DH12'
destiny_attention_df1_weekday['COD.9']='DH02'
destiny_attention_df1_weekday['COD.10']='DH11'
destiny_attention_df1_weekday['COD.11']='DH04'
destiny_attention_df1_weekday['COD.12']='DH06'
destiny_attention_df1_weekday['COD.13']='DH07'
destiny_attention_df1_weekday['COD.14']='DH00'

Dest_dict={'DH01':'La Granja','DH09':'Comayagua','DH03':'Choluteca','DH15':'Juticalpa','DH10':'Talanga','DH08':'Danli',
          'DH13':'La Entrada','DH05':'Santa Rosa','DH12':'Santa Barbara','DH02':'La Ceiba','DH11':'Saba','DH04':'Puerto Cortes','DH06':'Progreso',
          'DH07':'Tela','DH00':'San Pedro Sula'}
print(Dest_dict)

destiny_attention_df_weekday=destiny_attention_df1_weekday[['Inicio','Fin','T1','COD']].append(destiny_attention_df1_weekday[['Inicio.1','Fin.1','T1.1','COD.1']].rename(columns={'Inicio.1':'Inicio','Fin.1':'Fin','T1.1':'T1','COD.1':'COD'})).append(destiny_attention_df1_weekday[['Inicio.2','Fin.2','T1.2','COD.2']].rename(columns={'Inicio.2':'Inicio','Fin.2':'Fin','T1.2':'T1','COD.2':'COD'})).append(destiny_attention_df1_weekday[['Inicio.3','Fin.3','T1.3','COD.3']].rename(columns={'Inicio.3':'Inicio','Fin.3':'Fin','T1.3':'T1','COD.3':'COD'})).append(destiny_attention_df1_weekday[['Inicio.4','Fin.4','T1.4','COD.4']].rename(columns={'Inicio.4':'Inicio','Fin.4':'Fin','T1.4':'T1','COD.4':'COD'})).append(destiny_attention_df1_weekday[['Inicio.5','Fin.5','T1.5','COD.5']].rename(columns={'Inicio.5':'Inicio','Fin.5':'Fin','T1.5':'T1','COD.5':'COD'})).append(destiny_attention_df1_weekday[['Inicio.6','Fin.6','T1.6','COD.6']].rename(columns={'Inicio.6':'Inicio','Fin.6':'Fin','T1.6':'T1','COD.6':'COD'})).append(destiny_attention_df1_weekday[['Inicio.7','Fin.7','T1.7','COD.7']].rename(columns={'Inicio.7':'Inicio','Fin.7':'Fin','T1.7':'T1','COD.7':'COD'})).append(destiny_attention_df1_weekday[['Inicio.8','Fin.8','T1.8','COD.8']].rename(columns={'Inicio.8':'Inicio','Fin.8':'Fin','T1.8':'T1','COD.8':'COD'})).append(destiny_attention_df1_weekday[['Inicio.9','Fin.9','T1.9','COD.9']].rename(columns={'Inicio.9':'Inicio','Fin.9':'Fin','T1.9':'T1','COD.9':'COD'})).append(destiny_attention_df1_weekday[['Inicio.10','Fin.10','T1.10','COD.10']].rename(columns={'Inicio.10':'Inicio','Fin.10':'Fin','T1.10':'T1','COD.10':'COD'})).append(destiny_attention_df1_weekday[['Inicio.11','Fin.11','T1.11','COD.11']].rename(columns={'Inicio.11':'Inicio','Fin.11':'Fin','T1.11':'T1','COD.11':'COD'})).append(destiny_attention_df1_weekday[['Inicio.12','Fin.12','T1.12','COD.12']].rename(columns={'Inicio.12':'Inicio','Fin.12':'Fin','T1.12':'T1','COD.12':'COD'})).append(destiny_attention_df1_weekday[['Inicio.13','Fin.13','T1.13','COD.13']].rename(columns={'Inicio.13':'Inicio','Fin.13':'Fin','T1.13':'T1','COD.13':'COD'})).append(destiny_attention_df1_weekday[['Inicio.14','Fin.14','T1.14','COD.14']].rename(columns={'Inicio.14':'Inicio','Fin.14':'Fin','T1.14':'T1','COD.14':'COD'}))
destiny_attention_df_weekday['CDD']=destiny_attention_df_weekday['COD'].map(Dest_dict)

destiny_attention_df_weekday=destiny_attention_df_weekday.rename(columns={'Inicio':'Attention_Start','Fin':'Attention_End','T1':'Weekday_attention'})
destiny_attention_df_weekday=destiny_attention_df_weekday.dropna(subset=['Attention_Start'])
destiny_attention_df_weekday=destiny_attention_df_weekday.fillna(0)
destiny_attention_df_weekday=destiny_attention_df_weekday.reset_index()

destiny_attention_df_weekday['Attention_Start']=destiny_attention_df_weekday['Attention_Start'].astype(str).apply(format1)
destiny_attention_df_weekday['Attention_End']=destiny_attention_df_weekday['Attention_End'].astype(str).apply(format1)  
destiny_attention_df_weekday['Attention_Start'] = pd.to_datetime(destiny_attention_df_weekday['Attention_Start'], format='%H:%M:%S').dt.time
destiny_attention_df_weekday['Attention_End'] = pd.to_datetime(destiny_attention_df_weekday['Attention_End'], format='%H:%M:%S').dt.time
destiny_attention_df_weekday=destiny_attention_df_weekday[['COD','CDD','Attention_Start','Attention_End','Weekday_attention']]

{'DH01': 'La Granja', 'DH09': 'Comayagua', 'DH03': 'Choluteca', 'DH15': 'Juticalpa', 'DH10': 'Talanga', 'DH08': 'Danli', 'DH13': 'La Entrada', 'DH05': 'Santa Rosa', 'DH12': 'Santa Barbara', 'DH02': 'La Ceiba', 'DH11': 'Saba', 'DH04': 'Puerto Cortes', 'DH06': 'Progreso', 'DH07': 'Tela', 'DH00': 'San Pedro Sula'}


In [40]:
destiny_attention_df1_Sat=destiny_attention_df1_Sat.drop(['Unnamed: 4','Unnamed: 9','Unnamed: 14','Unnamed: 19','Unnamed: 24',
                        'Unnamed: 29','Unnamed: 34','Unnamed: 39','Unnamed: 44','Unnamed: 49','Unnamed: 54','Unnamed: 59','Unnamed: 64','Unnamed: 69'],axis=1)
destiny_attention_df1_Sat['COD']='DH01'
destiny_attention_df1_Sat['COD.1']='DH09'
destiny_attention_df1_Sat['COD.2']='DH03'
destiny_attention_df1_Sat['COD.3']='DH15'
destiny_attention_df1_Sat['COD.4']='DH10'
destiny_attention_df1_Sat['COD.5']='DH08'
destiny_attention_df1_Sat['COD.6']='DH13'
destiny_attention_df1_Sat['COD.7']='DH05'
destiny_attention_df1_Sat['COD.8']='DH12'
destiny_attention_df1_Sat['COD.9']='DH02'
destiny_attention_df1_Sat['COD.10']='DH11'
destiny_attention_df1_Sat['COD.11']='DH04'
destiny_attention_df1_Sat['COD.12']='DH06'
destiny_attention_df1_Sat['COD.13']='DH07'
destiny_attention_df1_Sat['COD.14']='DH00'

Dest_dict={'DH01':'La Granja','DH09':'Comayagua','DH03':'Choluteca','DH15':'Juticalpa','DH10':'Talanga','DH08':'Danli',
          'DH13':'La Entrada','DH05':'Santa Rosa','DH12':'Santa Barbara','DH02':'La Ceiba','DH11':'Saba','DH04':'Puerto Cortes','DH06':'Progreso',
          'DH07':'Tela','DH00':'San Pedro Sula'}
print(Dest_dict)

destiny_attention_df_Sat=destiny_attention_df1_Sat[['Inicio','Fin','T1','COD']].append(destiny_attention_df1_Sat[['Inicio.1','Fin.1','T1.1','COD.1']].rename(columns={'Inicio.1':'Inicio','Fin.1':'Fin','T1.1':'T1','COD.1':'COD'})).append(destiny_attention_df1_Sat[['Inicio.2','Fin.2','T1.2','COD.2']].rename(columns={'Inicio.2':'Inicio','Fin.2':'Fin','T1.2':'T1','COD.2':'COD'})).append(destiny_attention_df1_Sat[['Inicio.3','Fin.3','T1.3','COD.3']].rename(columns={'Inicio.3':'Inicio','Fin.3':'Fin','T1.3':'T1','COD.3':'COD'})).append(destiny_attention_df1_Sat[['Inicio.4','Fin.4','T1.4','COD.4']].rename(columns={'Inicio.4':'Inicio','Fin.4':'Fin','T1.4':'T1','COD.4':'COD'})).append(destiny_attention_df1_Sat[['Inicio.5','Fin.5','T1.5','COD.5']].rename(columns={'Inicio.5':'Inicio','Fin.5':'Fin','T1.5':'T1','COD.5':'COD'})).append(destiny_attention_df1_Sat[['Inicio.6','Fin.6','T1.6','COD.6']].rename(columns={'Inicio.6':'Inicio','Fin.6':'Fin','T1.6':'T1','COD.6':'COD'})).append(destiny_attention_df1_Sat[['Inicio.7','Fin.7','T1.7','COD.7']].rename(columns={'Inicio.7':'Inicio','Fin.7':'Fin','T1.7':'T1','COD.7':'COD'})).append(destiny_attention_df1_Sat[['Inicio.8','Fin.8','T1.8','COD.8']].rename(columns={'Inicio.8':'Inicio','Fin.8':'Fin','T1.8':'T1','COD.8':'COD'})).append(destiny_attention_df1_Sat[['Inicio.9','Fin.9','T1.9','COD.9']].rename(columns={'Inicio.9':'Inicio','Fin.9':'Fin','T1.9':'T1','COD.9':'COD'})).append(destiny_attention_df1_Sat[['Inicio.10','Fin.10','T1.10','COD.10']].rename(columns={'Inicio.10':'Inicio','Fin.10':'Fin','T1.10':'T1','COD.10':'COD'})).append(destiny_attention_df1_Sat[['Inicio.11','Fin.11','T1.11','COD.11']].rename(columns={'Inicio.11':'Inicio','Fin.11':'Fin','T1.11':'T1','COD.11':'COD'})).append(destiny_attention_df1_Sat[['Inicio.12','Fin.12','T1.12','COD.12']].rename(columns={'Inicio.12':'Inicio','Fin.12':'Fin','T1.12':'T1','COD.12':'COD'})).append(destiny_attention_df1_Sat[['Inicio.13','Fin.13','T1.13','COD.13']].rename(columns={'Inicio.13':'Inicio','Fin.13':'Fin','T1.13':'T1','COD.13':'COD'})).append(destiny_attention_df1_Sat[['Inicio.14','Fin.14','T1.14','COD.14']].rename(columns={'Inicio.14':'Inicio','Fin.14':'Fin','T1.14':'T1','COD.14':'COD'}))
destiny_attention_df_Sat['CDD']=destiny_attention_df_Sat['COD'].map(Dest_dict)

destiny_attention_df_Sat=destiny_attention_df_Sat.rename(columns={'Inicio':'Attention_Start','Fin':'Attention_End','T1':'Sat_attention'})
destiny_attention_df_Sat=destiny_attention_df_Sat.dropna(subset=['Attention_Start'])
destiny_attention_df_Sat=destiny_attention_df_Sat.fillna(0)
destiny_attention_df_Sat=destiny_attention_df_Sat.reset_index()

destiny_attention_df_Sat['Attention_Start']=destiny_attention_df_Sat['Attention_Start'].astype(str).apply(format1)
destiny_attention_df_Sat['Attention_End']=destiny_attention_df_Sat['Attention_End'].astype(str).apply(format1)  
destiny_attention_df_Sat['Attention_Start'] = pd.to_datetime(destiny_attention_df_Sat['Attention_Start'], format='%H:%M:%S').dt.time
destiny_attention_df_Sat['Attention_End'] = pd.to_datetime(destiny_attention_df_Sat['Attention_End'], format='%H:%M:%S').dt.time
destiny_attention_df_Sat=destiny_attention_df_Sat[['COD','CDD','Attention_Start','Attention_End','Sat_attention']]

{'DH01': 'La Granja', 'DH09': 'Comayagua', 'DH03': 'Choluteca', 'DH15': 'Juticalpa', 'DH10': 'Talanga', 'DH08': 'Danli', 'DH13': 'La Entrada', 'DH05': 'Santa Rosa', 'DH12': 'Santa Barbara', 'DH02': 'La Ceiba', 'DH11': 'Saba', 'DH04': 'Puerto Cortes', 'DH06': 'Progreso', 'DH07': 'Tela', 'DH00': 'San Pedro Sula'}


In [41]:
destiny_attention_df1_Sun.columns

Index(['Turno', 'Inicio', 'Fin', 'T1', 'Unnamed: 4', 'Turno.1', 'Inicio.1',
       'Fin.1', 'T1.1', 'Unnamed: 9', 'Turno.2', 'Inicio.2', 'Fin.2', 'T1.2',
       'Unnamed: 14', 'Turno.3', 'Inicio.3', 'Fin.3', 'T1.3', 'Unnamed: 19',
       'Turno.4', 'Inicio.4', 'Fin.4', 'T1.4', 'Unnamed: 24', 'Turno.5',
       'Inicio.5', 'Fin.5', 'T1.5', 'Unnamed: 29', 'Turno.6', 'Inicio.6',
       'Fin.6', 'T1.6', 'Unnamed: 34', 'Turno.7', 'Inicio.7', 'Fin.7', 'T1.7',
       'Unnamed: 39', 'Turno.8', 'Inicio.8', 'Fin.8', 'T1.8', 'Unnamed: 44',
       'Turno.9', 'Inicio.9', 'Fin.9', 'T1.9', 'Unnamed: 49', 'Turno.10',
       'Inicio.10', 'Fin.10', 'T1.10', 'Unnamed: 54', 'Turno.11', 'Inicio.11',
       'Fin.11', 'T1.11', 'Unnamed: 59', 'Turno.12', 'Inicio.12', 'Fin.12',
       'T1.12', 'Unnamed: 64', 'Turno.13', 'Inicio.13', 'Fin.13', 'T1.13',
       'Unnamed: 69', 'Unnamed: 70', 'Unnamed: 71', 'Unnamed: 72',
       'Unnamed: 73'],
      dtype='object')

In [42]:
destiny_attention_df1_Sun=destiny_attention_df1_Sun.drop(['Unnamed: 4','Unnamed: 9','Unnamed: 14','Unnamed: 19','Unnamed: 24',
                        'Unnamed: 29','Unnamed: 34','Unnamed: 39','Unnamed: 44','Unnamed: 49','Unnamed: 54','Unnamed: 59','Unnamed: 64','Unnamed: 69','Unnamed: 70', 'Unnamed: 71', 'Unnamed: 72',
       'Unnamed: 73'],axis=1)
destiny_attention_df1_Sun['COD']='DH01'
destiny_attention_df1_Sun['COD.1']='DH09'
destiny_attention_df1_Sun['COD.2']='DH03'
destiny_attention_df1_Sun['COD.3']='DH15'
destiny_attention_df1_Sun['COD.4']='DH10'
destiny_attention_df1_Sun['COD.5']='DH08'
destiny_attention_df1_Sun['COD.6']='DH13'
destiny_attention_df1_Sun['COD.7']='DH05'
destiny_attention_df1_Sun['COD.8']='DH12'
destiny_attention_df1_Sun['COD.9']='DH02'
destiny_attention_df1_Sun['COD.10']='DH11'
destiny_attention_df1_Sun['COD.11']='DH04'
destiny_attention_df1_Sun['COD.12']='DH06'
destiny_attention_df1_Sun['COD.13']='DH00'

Dest_dict={'DH01':'La Granja','DH09':'Comayagua','DH03':'Choluteca','DH15':'Juticalpa','DH10':'Talanga','DH08':'Danli',
          'DH13':'La Entrada','DH05':'Santa Rosa','DH12':'Santa Barbara','DH02':'La Ceiba','DH11':'Saba','DH04':'Puerto Cortes','DH06':'Progreso',
          'DH00':'San Pedro Sula'}
print(Dest_dict)

destiny_attention_df_Sun=destiny_attention_df1_Sun[['Inicio','Fin','T1','COD']].append(destiny_attention_df1_Sun[['Inicio.1','Fin.1','T1.1','COD.1']].rename(columns={'Inicio.1':'Inicio','Fin.1':'Fin','T1.1':'T1','COD.1':'COD'})).append(destiny_attention_df1_Sun[['Inicio.2','Fin.2','T1.2','COD.2']].rename(columns={'Inicio.2':'Inicio','Fin.2':'Fin','T1.2':'T1','COD.2':'COD'})).append(destiny_attention_df1_Sun[['Inicio.3','Fin.3','T1.3','COD.3']].rename(columns={'Inicio.3':'Inicio','Fin.3':'Fin','T1.3':'T1','COD.3':'COD'})).append(destiny_attention_df1_Sun[['Inicio.4','Fin.4','T1.4','COD.4']].rename(columns={'Inicio.4':'Inicio','Fin.4':'Fin','T1.4':'T1','COD.4':'COD'})).append(destiny_attention_df1_Sun[['Inicio.5','Fin.5','T1.5','COD.5']].rename(columns={'Inicio.5':'Inicio','Fin.5':'Fin','T1.5':'T1','COD.5':'COD'})).append(destiny_attention_df1_Sun[['Inicio.6','Fin.6','T1.6','COD.6']].rename(columns={'Inicio.6':'Inicio','Fin.6':'Fin','T1.6':'T1','COD.6':'COD'})).append(destiny_attention_df1_Sun[['Inicio.7','Fin.7','T1.7','COD.7']].rename(columns={'Inicio.7':'Inicio','Fin.7':'Fin','T1.7':'T1','COD.7':'COD'})).append(destiny_attention_df1_Sun[['Inicio.8','Fin.8','T1.8','COD.8']].rename(columns={'Inicio.8':'Inicio','Fin.8':'Fin','T1.8':'T1','COD.8':'COD'})).append(destiny_attention_df1_Sun[['Inicio.9','Fin.9','T1.9','COD.9']].rename(columns={'Inicio.9':'Inicio','Fin.9':'Fin','T1.9':'T1','COD.9':'COD'})).append(destiny_attention_df1_Sun[['Inicio.10','Fin.10','T1.10','COD.10']].rename(columns={'Inicio.10':'Inicio','Fin.10':'Fin','T1.10':'T1','COD.10':'COD'})).append(destiny_attention_df1_Sun[['Inicio.11','Fin.11','T1.11','COD.11']].rename(columns={'Inicio.11':'Inicio','Fin.11':'Fin','T1.11':'T1','COD.11':'COD'})).append(destiny_attention_df1_Sun[['Inicio.12','Fin.12','T1.12','COD.12']].rename(columns={'Inicio.12':'Inicio','Fin.12':'Fin','T1.12':'T1','COD.12':'COD'})).append(destiny_attention_df1_Sun[['Inicio.13','Fin.13','T1.13','COD.13']].rename(columns={'Inicio.13':'Inicio','Fin.13':'Fin','T1.13':'T1','COD.13':'COD'}))
destiny_attention_df_Sun['CDD']=destiny_attention_df_Sun['COD'].map(Dest_dict)

destiny_attention_df_Sun=destiny_attention_df_Sun.rename(columns={'Inicio':'Attention_Start','Fin':'Attention_End','T1':'Sun_attention'})
destiny_attention_df_Sun=destiny_attention_df_Sun.dropna(subset=['Attention_Start'])
destiny_attention_df_Sun=destiny_attention_df_Sun.fillna(0)
destiny_attention_df_Sun=destiny_attention_df_Sun.reset_index()

destiny_attention_df_Sun['Attention_Start']=destiny_attention_df_Sun['Attention_Start'].astype(str).apply(format1)
destiny_attention_df_Sun['Attention_End']=destiny_attention_df_Sun['Attention_End'].astype(str).apply(format1)  
destiny_attention_df_Sun['Attention_Start'] = pd.to_datetime(destiny_attention_df_Sun['Attention_Start'], format='%H:%M:%S').dt.time
destiny_attention_df_Sun['Attention_End'] = pd.to_datetime(destiny_attention_df_Sun['Attention_End'], format='%H:%M:%S').dt.time
destiny_attention_df_Sun=destiny_attention_df_Sun[['COD','CDD','Attention_Start','Attention_End','Sun_attention']]

{'DH01': 'La Granja', 'DH09': 'Comayagua', 'DH03': 'Choluteca', 'DH15': 'Juticalpa', 'DH10': 'Talanga', 'DH08': 'Danli', 'DH13': 'La Entrada', 'DH05': 'Santa Rosa', 'DH12': 'Santa Barbara', 'DH02': 'La Ceiba', 'DH11': 'Saba', 'DH04': 'Puerto Cortes', 'DH06': 'Progreso', 'DH00': 'San Pedro Sula'}


In [43]:
destiny_attention_df=destiny_attention_df_weekday.merge(destiny_attention_df_Sat, on=['COD','CDD','Attention_Start','Attention_End'],how='left').merge(destiny_attention_df_Sun, on=['COD','CDD','Attention_Start','Attention_End'],how='left')
destiny_attention_df=destiny_attention_df.fillna(0)

In [44]:
destiny_attention_df.columns

Index(['COD', 'CDD', 'Attention_Start', 'Attention_End', 'Weekday_attention',
       'Sat_attention', 'Sun_attention'],
      dtype='object')

#### Destination - Brewery - Customer

In [45]:
destiny_attention_Custdf1_weekday =pd.read_excel('Data/MASTER DATA VENTANA DE ATENCION T2.xlsx',sheet_name='Ventanas de ATENCION - Customer',skiprows=1)
destiny_attention_Custdf1_weekday=destiny_attention_Custdf1_weekday.head(n=24)

destiny_attention_Custdf1_weekday=destiny_attention_Custdf1_weekday.drop(['Unnamed: 4','Unnamed: 9','Unnamed: 14','Unnamed: 19','Unnamed: 24',
                        'Unnamed: 29','Unnamed: 34','Unnamed: 39','Unnamed: 44','Unnamed: 49','Unnamed: 54','Unnamed: 59','Unnamed: 64','Unnamed: 69','Unnamed: 74','Unnamed: 79','Unnamed: 84','Unnamed: 89','Unnamed: 94','Unnamed: 99'],axis=1)

destiny_attention_Custdf1_weekday['COD']='AQ'
destiny_attention_Custdf1_weekday['COD.1']='CK'
destiny_attention_Custdf1_weekday['COD.2']='AO'
destiny_attention_Custdf1_weekday['COD.3']='CP'
destiny_attention_Custdf1_weekday['COD.4']='HI'
destiny_attention_Custdf1_weekday['COD.5']='AH'
destiny_attention_Custdf1_weekday['COD.6']='ID'
destiny_attention_Custdf1_weekday['COD.7']='BT'
destiny_attention_Custdf1_weekday['COD.8']='BX'
destiny_attention_Custdf1_weekday['COD.9']='CD'
destiny_attention_Custdf1_weekday['COD.10']='CJ'
destiny_attention_Custdf1_weekday['COD.11']='AK'
destiny_attention_Custdf1_weekday['COD.12']='AR'
destiny_attention_Custdf1_weekday['COD.13']='AC'
destiny_attention_Custdf1_weekday['COD.14']='BP'
destiny_attention_Custdf1_weekday['COD.15']='AI'
destiny_attention_Custdf1_weekday['COD.16']='AT'
destiny_attention_Custdf1_weekday['COD.17']='SPS'
destiny_attention_Custdf1_weekday['COD.18']='BM'
destiny_attention_Custdf1_weekday['COD.19']='CRUZ AH'
destiny_attention_Custdf1_weekday['COD.20']='PAZ'

Dest_dict_Cust={'AQ':' ANA LUISA MARTINEZ AQ','CK':' CROSSD YORO CK','AO':' DANIEL RAMIREZ AO','CP':' DEPOSITO EL PUNTO CP','HI':' DEPOSITO MARROQUIN HI','AH':' GASPAR PINEDA AH',
          'ID':' GUALACO ID','BT':' IRMA ALVARADO BT','BX':' MARCO TULIO MONTALBAN BX','CD':' MILO MUNGUIA CD','CJ':' PATUCA CJ','AK':' RIGOBERTO DEL CID AK','AR':' ROBERTO JEFF AR',
          'AC':' RODOLFO DIAZ AC','BP':' SALVADOR ROBLES BP','AI':' TAULABE AI','AT':' TOLEDO AT','SPS':' WALMART SPS','BM':' BODEGUITA MURILLO BM','CRUZ AH':' GASPAR PINEDA SANTA CRUZ AH',
          'PAZ':' SAN FRANCISCO DE LA PAZ'}

destiny_attention_Custdf_weekday=destiny_attention_Custdf1_weekday[['Inicio','Fin','T1','COD']].append(destiny_attention_Custdf1_weekday[['Inicio.1','Fin.1','T1.1','COD.1']].rename(columns={'Inicio.1':'Inicio','Fin.1':'Fin','T1.1':'T1','COD.1':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.2','Fin.2','T1.2','COD.2']].rename(columns={'Inicio.2':'Inicio','Fin.2':'Fin','T1.2':'T1','COD.2':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.3','Fin.3','T1.3','COD.3']].rename(columns={'Inicio.3':'Inicio','Fin.3':'Fin','T1.3':'T1','COD.3':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.4','Fin.4','T1.4','COD.4']].rename(columns={'Inicio.4':'Inicio','Fin.4':'Fin','T1.4':'T1','COD.4':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.5','Fin.5','T1.5','COD.5']].rename(columns={'Inicio.5':'Inicio','Fin.5':'Fin','T1.5':'T1','COD.5':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.6','Fin.6','T1.6','COD.6']].rename(columns={'Inicio.6':'Inicio','Fin.6':'Fin','T1.6':'T1','COD.6':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.7','Fin.7','T1.7','COD.7']].rename(columns={'Inicio.7':'Inicio','Fin.7':'Fin','T1.7':'T1','COD.7':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.8','Fin.8','T1.8','COD.8']].rename(columns={'Inicio.8':'Inicio','Fin.8':'Fin','T1.8':'T1','COD.8':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.9','Fin.9','T1.9','COD.9']].rename(columns={'Inicio.9':'Inicio','Fin.9':'Fin','T1.9':'T1','COD.9':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.10','Fin.10','T1.10','COD.10']].rename(columns={'Inicio.10':'Inicio','Fin.10':'Fin','T1.10':'T1','COD.10':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.11','Fin.11','T1.11','COD.11']].rename(columns={'Inicio.11':'Inicio','Fin.11':'Fin','T1.11':'T1','COD.11':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.12','Fin.12','T1.12','COD.12']].rename(columns={'Inicio.12':'Inicio','Fin.12':'Fin','T1.12':'T1','COD.12':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.13','Fin.13','T1.13','COD.13']].rename(columns={'Inicio.13':'Inicio','Fin.13':'Fin','T1.13':'T1','COD.13':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.14','Fin.14','T1.14','COD.14']].rename(columns={'Inicio.14':'Inicio','Fin.14':'Fin','T1.14':'T1','COD.14':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.15','Fin.15','T1.15','COD.15']].rename(columns={'Inicio.15':'Inicio','Fin.15':'Fin','T1.15':'T1','COD.15':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.16','Fin.16','T1.16','COD.16']].rename(columns={'Inicio.16':'Inicio','Fin.16':'Fin','T1.16':'T1','COD.16':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.17','Fin.17','T1.17','COD.17']].rename(columns={'Inicio.17':'Inicio','Fin.17':'Fin','T1.17':'T1','COD.17':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.18','Fin.18','T1.18','COD.18']].rename(columns={'Inicio.18':'Inicio','Fin.18':'Fin','T1.18':'T1','COD.18':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.19','Fin.19','T1.19','COD.19']].rename(columns={'Inicio.19':'Inicio','Fin.19':'Fin','T1.19':'T1','COD.19':'COD'})).append(destiny_attention_Custdf1_weekday[['Inicio.20','Fin.20','T1.20','COD.20']].rename(columns={'Inicio.20':'Inicio','Fin.20':'Fin','T1.20':'T1','COD.20':'COD'}))
destiny_attention_Custdf_weekday['CDD']=destiny_attention_Custdf_weekday['COD'].map(Dest_dict_Cust)

destiny_attention_Custdf_weekday=destiny_attention_Custdf_weekday.rename(columns={'Inicio':'Attention_Start','Fin':'Attention_End','T1':'Weekday_attention'})
destiny_attention_Custdf_weekday=destiny_attention_Custdf_weekday.dropna(subset=['Attention_Start'])
destiny_attention_Custdf_weekday=destiny_attention_Custdf_weekday.fillna(0)
destiny_attention_Custdf_weekday=destiny_attention_Custdf_weekday.reset_index()

destiny_attention_Custdf_weekday['Attention_Start']=destiny_attention_Custdf_weekday['Attention_Start'].astype(str).apply(format1)
destiny_attention_Custdf_weekday['Attention_End']=destiny_attention_Custdf_weekday['Attention_End'].astype(str).apply(format1)  
destiny_attention_Custdf_weekday['Attention_Start'] = pd.to_datetime(destiny_attention_Custdf_weekday['Attention_Start'], format='%H:%M:%S').dt.time
destiny_attention_Custdf_weekday['Attention_End'] = pd.to_datetime(destiny_attention_Custdf_weekday['Attention_End'], format='%H:%M:%S').dt.time
destiny_attention_Custdf_weekday=destiny_attention_Custdf_weekday[['COD','CDD','Attention_Start','Attention_End','Weekday_attention']]

In [46]:
destiny_attention_Custdf=destiny_attention_Custdf_weekday
destiny_attention_Custdf['Sat_attention']=destiny_attention_Custdf['Weekday_attention']
destiny_attention_Custdf['Sun_attention']=0
destiny_attention_Custdf=destiny_attention_Custdf.fillna(0)

In [47]:
#destiny_attention_Custdf

#### Time taken

In [48]:
def convert_hr_min_to_minutes(str1):
    ## 04:30	 to 270 mins
    hour1=str1.split(":")[0]
    min1=str1.split(":")[1]
    hour1_min=int(hour1)*60
    total_min=hour1_min+int(min1)
    return total_min

In [49]:
Time_df =pd.read_excel('Data/Inputs - Static.xlsx',sheet_name='Tiempos',skiprows=2)
Time_df_cust =pd.read_excel('Data/Inputs - Static.xlsx',sheet_name='Tiempos_Cust',skiprows=2)

In [50]:
### For 12 columns 

############ Brewery to DC
distance_and_time =pd.DataFrame(index=np.arange(1), columns=np.arange(12))
distance_and_time=distance_and_time.rename(columns={0: 'Origin',1: 'Origin_Code',2: 'Destiny',3: 'Destiny_Code',4: 'Clave',5:'time_one_way_min',6:'time_two_way_min',7:'Loading_time_origin_min',8:'unloading_loading_time_in_dest_min',9:'row_time_origin_min',10:'waiting_time_origin_min',11:'break_time_min'})

############ Brewery to Customer
distance_and_time_cust =pd.DataFrame(index=np.arange(1), columns=np.arange(12))
distance_and_time_cust=distance_and_time_cust.rename(columns={0: 'Origin',1: 'Origin_Code',2: 'Destiny',3: 'Destiny_Code',4: 'Clave',5:'time_one_way_min',6:'time_two_way_min',7:'Loading_time_origin_min',8:'unloading_loading_time_in_dest_min',9:'row_time_origin_min',10:'waiting_time_origin_min',11:'break_time_min'})

##### Put the Individual list in time itself

origin_dict={'SH01':'CSD','DH22':'KEYMART','DH20':'POLVORIN','BH01':'BEER','DH19':'ALDESA'}

#### Dest Brewery to DC
Dest_dict={'DH01':'La Granja','DH09':'Comayagua','DH03':'Choluteca','DH15':'Juticalpa','DH10':'Talanga','DH08':'Danli',
          'DH13':'La Entrada','DH05':'Santa Rosa','DH12':'Santa Barbara','DH02':'La Ceiba','DH11':'Saba','DH04':'Puerto Cortes','DH06':'Progreso',
          'DH00':'San Pedro Sula','DH07':'Tela','DH14':'Roatan','DH03-I':'Choluteca Individual','DH10-I':'Talanga Individual','DH15-I':'Juticalpa Individual'}

#### Dest Brewery to Customer
Dest_dict_Cust={'AQ':' ANA LUISA MARTINEZ AQ','CK':' CROSSD YORO CK','AO':' DANIEL RAMIREZ AO','CP':' DEPOSITO EL PUNTO CP','HI':' DEPOSITO MARROQUIN HI','AH':' GASPAR PINEDA AH',
          'ID':' GUALACO ID','BT':' IRMA ALVARADO BT','BX':' MARCO TULIO MONTALBAN BX','CD':' MILO MUNGUIA CD','CJ':' PATUCA CJ','AK':' RIGOBERTO DEL CID AK','AR':' ROBERTO JEFF AR',
          'AC':' RODOLFO DIAZ AC','BP':' SALVADOR ROBLES BP','AI':' TAULABE AI','AT':' TOLEDO AT','SPS':' WALMART SPS','BM':' BODEGUITA MURILLO BM','CRUZ AH':' GASPAR PINEDA SANTA CRUZ AH',
          'PAZ':' SAN FRANCISCO DE LA PAZ'}

############ Brewery to DC
distance_and_time1 =pd.DataFrame()
for i in list(origin_dict.keys()):
    distance_and_time['Origin']=origin_dict[i]
    distance_and_time['Origin_Code']=i
    for j in list(Dest_dict.keys()):
        distance_and_time['Destiny']=Dest_dict[j]
        distance_and_time['Destiny_Code']=j
        distance_and_time1=distance_and_time1.append(distance_and_time)
        
distance_and_time1['Clave']= distance_and_time1['Origin'].str.upper()+"-"+distance_and_time1['Destiny'].str.upper()
distance_and_time1=distance_and_time1.fillna(0)

distance_and_time2 =pd.DataFrame()
for i in range(Time_df.shape[0]):
    #print(i)
    time_dest_temp=distance_and_time1[distance_and_time1['Destiny']==Time_df.iloc[i]['Unnamed: 0']]
    time_dest_temp['waiting_time_origin_min']=Time_df.iloc[i]['TC']
    time_dest_temp['Loading_time_origin_min']=Time_df.iloc[i]['TP']
    time_dest_temp['row_time_origin_min']=Time_df.iloc[i]['TF']
    time_dest_temp['time_one_way_min']=Time_df.iloc[i]['TT']
    time_dest_temp['unloading_loading_time_in_dest_min']=Time_df.iloc[i]['TCD']
    time_dest_temp['break_time_min']=Time_df.iloc[i]['TD']
    time_dest_temp['time_two_way_min']=Time_df.iloc[i]['TR']
    distance_and_time2=distance_and_time2.append(time_dest_temp)

############ Brewery to Customer
distance_and_time1_cust =pd.DataFrame()
for i in list(origin_dict.keys()):
    distance_and_time_cust['Origin']=origin_dict[i]
    distance_and_time_cust['Origin_Code']=i
    for j in list(Dest_dict_Cust.keys()):
        distance_and_time_cust['Destiny']=Dest_dict_Cust[j]
        distance_and_time_cust['Destiny_Code']=j
        distance_and_time1_cust=distance_and_time1_cust.append(distance_and_time_cust)
        
distance_and_time1_cust['Clave']= distance_and_time1_cust['Origin'].str.upper()+"-"+distance_and_time1_cust['Destiny'].str.upper()
distance_and_time1_cust=distance_and_time1_cust.fillna(0)

distance_and_time2_cust =pd.DataFrame()
for i in range(Time_df_cust.shape[0]):
    #print(i)
    time_dest_temp_cust=distance_and_time1_cust[distance_and_time1_cust['Destiny']==Time_df_cust.iloc[i]['Unnamed: 0']]
    time_dest_temp_cust['waiting_time_origin_min']=Time_df_cust.iloc[i]['TC']
    time_dest_temp_cust['Loading_time_origin_min']=Time_df_cust.iloc[i]['TP']
    time_dest_temp_cust['row_time_origin_min']=Time_df_cust.iloc[i]['TF']
    time_dest_temp_cust['time_one_way_min']=Time_df_cust.iloc[i]['TT']
    time_dest_temp_cust['unloading_loading_time_in_dest_min']=Time_df_cust.iloc[i]['TCD']
    time_dest_temp_cust['break_time_min']=Time_df_cust.iloc[i]['TD']
    time_dest_temp_cust['time_two_way_min']=Time_df_cust.iloc[i]['TR']
    distance_and_time2_cust=distance_and_time2_cust.append(time_dest_temp_cust)

    
time_cols=['time_one_way_min', 'time_two_way_min', 'Loading_time_origin_min',
       'unloading_loading_time_in_dest_min', 'row_time_origin_min',
       'waiting_time_origin_min', 'break_time_min']

############ Brewery to DC
for i in time_cols:
    #print(i)
    distance_and_time2[i]=distance_and_time2[i].astype(str).apply(format2)
    distance_and_time2[i]=distance_and_time2[i].astype(str).apply(convert_hr_min_to_minutes)
distance_and_time2['Loading_time_origin_min'] =distance_and_time2['waiting_time_origin_min']+distance_and_time2['Loading_time_origin_min']+distance_and_time2['row_time_origin_min']
distance_and_time=distance_and_time2

############ Brewery to Customer
for i in time_cols:
    #print(i)
    distance_and_time2_cust[i]=distance_and_time2_cust[i].astype(str).apply(format2)
    distance_and_time2_cust[i]=distance_and_time2_cust[i].astype(str).apply(convert_hr_min_to_minutes)
distance_and_time2_cust['Loading_time_origin_min'] =distance_and_time2_cust['waiting_time_origin_min']+distance_and_time2_cust['Loading_time_origin_min']+distance_and_time2_cust['row_time_origin_min']
distance_and_time_cust=distance_and_time2_cust

In [51]:
#distance_and_time

In [52]:
distance_and_time_cust['Destiny_Code'].unique()

array(['AT', 'AC', 'AH', 'AI', 'AK', 'AO', 'BX', 'AR', 'BP', 'BT', 'CD',
       'CJ', 'CK', 'CP', 'HI', 'ID', 'SPS', 'AQ', 'BM', 'CRUZ AH', 'PAZ'],
      dtype=object)

### Distribution Plan

#### Distribution plan - Brewery to DC

In [53]:
Dist_plan =pd.read_excel('Data/Space and Optimus DP_1409.xlsx')
Dist_plan['Fecha']=pd.to_datetime(Dist_plan['Fecha'])

Dist_plan1=Dist_plan[Dist_plan['Fecha']==pd.to_datetime(fecha_str, format='%d-%m-%Y')]
Dist_plan1=Dist_plan1.drop(['Fecha'],axis=1)
print(Dist_plan1.shape)
Dist_plan1=Dist_plan1.rename(columns={'Origen': 'Origin','Destino': 'Destination','Total': 'Total_boxes','sku_SAP': 'codigo'})
Dist_plan1 =Dist_plan1.merge(sku_description[['codigo','kg_weight_per_pallet','package_per_pallet']], on ='codigo', how ='left')
Dist_plan1['Total_pallets']=Dist_plan1['Total_boxes']/Dist_plan1['package_per_pallet']
Dist_plan1=Dist_plan1[Dist_plan1['Destination']!='DH14']


Dist_plan1['org-priority']=Dist_plan1['Origin']+"-"+Dist_plan1['Prioridad'].astype(str)
#print(Dist_plan1.head(50))
#code modified by Jitendra as sort is not happening properly as it is happening in excel sheet 
#commented 
# Dist_plan1=Dist_plan1.sort_values(['Prioridad'],ascending=[True])
#added 
Dist_plan1_1=Dist_plan1[(Dist_plan1.Prioridad==1)]
Dist_plan1_2=Dist_plan1[(Dist_plan1.Prioridad==2)]
Dist_plan1_3=Dist_plan1[(Dist_plan1.Prioridad==3)]
print(Dist_plan1.shape)
Dist_plan1=pd.concat([Dist_plan1_1,Dist_plan1_2,Dist_plan1_3])
print(Dist_plan1.shape)
#Question for Sejuti what to do if there is no priority for a row 
# print(Dist_plan1.head(10))
Dist_plan2 =Dist_plan1.copy(deep=True)

All_sku=list(Dist_plan1['codigo'].unique())

All_destiny=list(Dist_plan1['Destination'].unique())
print('total destiny',len(All_destiny))

(366, 8)
(343, 12)
(343, 12)
total destiny 15


#### Distribution plan - Brewery to Customer

In [54]:
Dist_plan_cust =pd.read_excel('Data/INPUTS SOCIOS ESTRATEGICOS.xlsx')
Dist_plan_cust['Fecha']=pd.to_datetime(Dist_plan_cust['Fecha'])

Dist_plan_cust1=Dist_plan_cust[Dist_plan_cust['Fecha']==pd.to_datetime(fecha_str, format='%d-%m-%Y')]
Dist_plan_cust1=Dist_plan_cust1.drop(['Fecha'],axis=1)
print(Dist_plan_cust1.shape)
Dist_plan_cust1=Dist_plan_cust1.rename(columns={'Origen': 'Origin','Destino': 'Destination','Ruta':'Route','Total': 'Total_boxes','sku_SAP': 'codigo'})
Dist_plan_cust1 =Dist_plan_cust1.merge(sku_description[['codigo','kg_weight_per_pallet','package_per_pallet']], on ='codigo', how ='left')
Dist_plan_cust1['Total_pallets']=Dist_plan_cust1['Total_boxes']/Dist_plan_cust1['package_per_pallet']
Dist_plan_cust1=Dist_plan_cust1[Dist_plan_cust1['Destination']!='DH14']


Dist_plan_cust1['org-priority']=Dist_plan_cust1['Origin']+"-"+Dist_plan_cust1['Prioridad'].astype(str)
#print(Dist_plan_cust1.head(50))
#code modified by Jitendra as sort is not happening properly as it is happening in excel sheet 
#commented 
Dist_plan_cust1=Dist_plan_cust1.sort_values(['Prioridad'],ascending=[True])
Dist_plan_cust1['Destination']=Dist_plan_cust1['Cod']

Dist_plan_cust1_1=Dist_plan_cust1[(Dist_plan_cust1.Prioridad==1)]
Dist_plan_cust1_2=Dist_plan_cust1[(Dist_plan_cust1.Prioridad==2)]
Dist_plan_cust1_3=Dist_plan_cust1[(Dist_plan_cust1.Prioridad==3)]
print(Dist_plan_cust1.shape)
Dist_plan_cust1=pd.concat([Dist_plan_cust1_1,Dist_plan_cust1_2,Dist_plan_cust1_3])
print(Dist_plan_cust1.shape)
#Question for Sejuti what to do if there is no priority for a row 
# print(Dist_plan_cust1.head(10))
Dist_plan_cust2 =Dist_plan_cust1.copy(deep=True)

All_sku_cust=list(Dist_plan_cust1['codigo'].unique())

All_destiny_cust=list(Dist_plan_cust1['Destination'].unique())
print('total destiny',len(All_destiny_cust))

(97, 8)
(97, 12)
(97, 12)
total destiny 9


In [55]:
print(Dist_plan_cust1.columns)
print(Dist_plan1.columns)

Index(['Cod', 'Origin', 'Destination', 'codigo', 'Prioridad', 'Total_boxes',
       'what is total unit', 'Route', 'kg_weight_per_pallet',
       'package_per_pallet', 'Total_pallets', 'org-priority'],
      dtype='object')
Index(['Origin', 'Destination', 'codigo', 'Prioridad', 'Total_boxes',
       'Box x Pallet', 'kg weight per pallet', 'pallets',
       'kg_weight_per_pallet', 'package_per_pallet', 'Total_pallets',
       'org-priority'],
      dtype='object')


#### Origin-Dest Before Melt DF for Jitendra

Brewery - DC

In [56]:
OrgPr1={}
for i in list(Dist_plan1['Origin'].unique()):
    #print(i)
    temp = Dist_plan1[Dist_plan1['Origin']==i]
    OrgPr1[i]=temp
    
OrgPrBefMelt1={}
for i in list(OrgPr1.keys()):
    #print(i)
    temp1=OrgPr1[i].pivot(index='codigo', columns='Destination', values='Total_pallets').fillna(0).reset_index().rename_axis(None, axis=1)
    OrgPrBefMelt1[i]=temp1
    #### Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of destinities-total 15.
    diff_col=list(set(All_destiny).difference(list(OrgPrBefMelt1[i].columns)))
    for l in diff_col:
        OrgPrBefMelt1[i][l]=0
        
# Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of SKUs.
for i in OrgPrBefMelt1.keys():
    OrgPrBefMelt1[i]=OrgPrBefMelt1[i].append(pd.DataFrame(list(set(All_sku).difference(list(OrgPrBefMelt1[i]['codigo'].unique()))), columns=['codigo'])).fillna(0).reset_index().drop(['index'],axis=1)
    OrgPrBefMelt1[i]=OrgPrBefMelt1[i].merge(sku_description[['codigo','kg_weight_per_pallet']])

In [57]:
for i in OrgPrBefMelt1.keys():
    print(OrgPrBefMelt1[i].shape)

(43, 17)
(43, 17)
(43, 17)


Brewery - Customers - **** Changing the code acc to team's need. We can club same destinations irrespective of origins

In [58]:
# OrgPr_C1={}
# for i in list(Dist_plan_cust1['Origin'].unique()):
#     #print(i)
#     temp_cust = Dist_plan_cust1[Dist_plan_cust1['Origin']==i]
#     OrgPr_C1[i]=temp_cust
    
# OrgPrCBefMelt1={}
# for i in list(OrgPr_C1.keys()):
#     #print(i)
#     temp_cust1=OrgPr_C1[i].pivot(index='codigo', columns='Destination', values='Total_pallets').fillna(0).reset_index().rename_axis(None, axis=1)
#     OrgPrCBefMelt1[i]=temp_cust1
#     #### Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of destinities-total 15.
#     diff_col=list(set(All_destiny_cust).difference(list(OrgPrCBefMelt1[i].columns)))
#     for l in diff_col:
#         OrgPrCBefMelt1[i][l]=0
        
# # Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of SKUs.
# for i in OrgPrCBefMelt1.keys():
#     OrgPrCBefMelt1[i]=OrgPrCBefMelt1[i].append(pd.DataFrame(list(set(All_sku_cust).difference(list(OrgPrCBefMelt1[i]['codigo'].unique()))), columns=['codigo'])).fillna(0).reset_index().drop(['index'],axis=1)
#     OrgPrCBefMelt1[i]=OrgPrCBefMelt1[i].merge(sku_description[['codigo','kg_weight_per_pallet']])
    

Dist_plan_cust1['One_Origin']='Honduras'

#Dist_plan_cust1=Dist_plan_cust1.reset_index().drop(['index'],axis=1)
# #             Dist_plan1=Dist_plan1.sort_values(['Prioridad'],ascending=[True])
OrgPr_C1={}
for i in list(Dist_plan_cust1['One_Origin'].unique()):
#print(i)
    temp = Dist_plan_cust1[Dist_plan_cust1['One_Origin']==i]
    OrgPr_C1[i]=temp

OrgPrCBefMelt1={}
for i in list(OrgPr_C1.keys()):
#print(i) 
    temp1=OrgPr_C1[i].pivot(index=['codigo','Origin'], columns='Destination', values='Total_pallets').fillna(0).reset_index().rename_axis(None, axis=1)
    OrgPrCBefMelt1[i]=temp1
#### Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of destinities-total 15.
    diff_col=list(set(All_destiny_cust).difference(list(OrgPrCBefMelt1[i].columns)))
    for l in diff_col:
        OrgPrCBefMelt1[i][l]=0

# Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of SKUs.
for p in OrgPrCBefMelt1.keys():
    OrgPrCBefMelt1[p]=OrgPrCBefMelt1[p].append(pd.DataFrame(list(set(All_sku_cust).difference(list(OrgPrCBefMelt1[p]['codigo'].unique()))), columns=['codigo'])).fillna(0).reset_index().drop(['index'],axis=1)
    OrgPrCBefMelt1[p]=OrgPrCBefMelt1[p].merge(sku_description[['codigo','kg_weight_per_pallet']])

In [59]:
for i in OrgPrCBefMelt1.keys():
    OrgPrCBefMelt1[i]=OrgPrCBefMelt1[i].append(pd.DataFrame(list(set(All_sku_cust).difference(list(OrgPrCBefMelt1[i]['codigo'].unique()))), columns=['codigo'])).fillna(0).reset_index().drop(['index'],axis=1)    

In [60]:
for i in OrgPrCBefMelt1.keys():
    print(OrgPrCBefMelt1[i].shape)

(53, 12)


In [61]:
#OrgPrCBefMelt1

#### Creating Origin Priority dataframe

Brewery - DC

In [62]:
OrgPr={}
for i in list(Dist_plan1['org-priority'].unique()):
    #print(i)
    temp = Dist_plan1[Dist_plan1['org-priority']==i]
    OrgPr[i]=temp
    
OrgPrBefMelt={}
for i in list(OrgPr.keys()):
    #print(i)
    temp1=OrgPr[i].pivot(index='codigo', columns='Destination', values='Total_pallets').fillna(0).reset_index().rename_axis(None, axis=1)
    OrgPrBefMelt[i]=temp1
    #### Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of destinities-total 15.
    diff_col=list(set(All_destiny).difference(list(OrgPrBefMelt[i].columns)))
    for l in diff_col:
        OrgPrBefMelt[i][l]=0
    
    
    # Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of SKUs.
for i in OrgPrBefMelt.keys():
    OrgPrBefMelt[i]=OrgPrBefMelt[i].append(pd.DataFrame(list(set(All_sku).difference(list(OrgPrBefMelt[i]['codigo'].unique()))), columns=['codigo'])).fillna(0).reset_index().drop(['index'],axis=1)

Brewery - Customer

In [63]:
OrgPrC={}
for i in list(Dist_plan_cust1['org-priority'].unique()):
    #print(i)
    temp_cust = Dist_plan_cust1[Dist_plan_cust1['org-priority']==i]
    OrgPrC[i]=temp_cust
    
OrgPrCBefMelt={}
for i in list(OrgPrC.keys()):
    #print(i)
    temp_cust1=OrgPrC[i].pivot(index='codigo', columns='Destination', values='Total_pallets').fillna(0).reset_index().rename_axis(None, axis=1)
    OrgPrCBefMelt[i]=temp_cust1
    #### Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of destinities-total 15.
    diff_col=list(set(All_destiny_cust).difference(list(OrgPrCBefMelt[i].columns)))
    for l in diff_col:
        OrgPrCBefMelt[i][l]=0
    
    
    # Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of SKUs.
for i in OrgPrCBefMelt.keys():
    OrgPrCBefMelt[i]=OrgPrCBefMelt[i].append(pd.DataFrame(list(set(All_sku_cust).difference(list(OrgPrCBefMelt[i]['codigo'].unique()))), columns=['codigo'])).fillna(0).reset_index().drop(['index'],axis=1)

In [64]:
OrgPrC={}
for i in list(Dist_plan_cust1['org-priority'].unique()):
    #print(i)
    temp_cust = Dist_plan_cust1[Dist_plan_cust1['org-priority']==i]
    OrgPrC[i]=temp_cust
    
OrgPrCBefMelt={}
for i in list(OrgPrC.keys()):
    #print(i)
    temp_cust1=OrgPrC[i].pivot(index='codigo', columns='Destination', values='Total_pallets').fillna(0).reset_index().rename_axis(None, axis=1)
    OrgPrCBefMelt[i]=temp_cust1
    #### Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of destinities-total 15.
    diff_col=list(set(All_destiny_cust).difference(list(OrgPrCBefMelt[i].columns)))
    for l in diff_col:
        OrgPrCBefMelt[i][l]=0
    
    
    # Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of SKUs.
for i in OrgPrCBefMelt.keys():
    OrgPrCBefMelt[i]=OrgPrCBefMelt[i].append(pd.DataFrame(list(set(All_sku_cust).difference(list(OrgPrCBefMelt[i]['codigo'].unique()))), columns=['codigo'])).fillna(0).reset_index().drop(['index'],axis=1)

### Origin Attention check

In [65]:
def get_attention_hour_list_v1(origin_attention_df,org,start_time_p,attention_list_allowed,attention_dict_allowed): 
    #get origin attention hour list which is 24 hr or max time time it can start from origin 
    
    origin_attention_df_1=origin_attention_df[origin_attention_df.COD==org]
    origin_attention_df_1=origin_attention_df_1.sort_values(by='Attention_Start')
    allowed_attention_list=attention_list_allowed
    allowed_attention_dict=attention_dict_allowed
    attention_hour_list=[]
    from itertools import cycle
    pool = cycle(origin_attention_df_1['Attention_Start'].values)
    start_time=start_time_p
    count=0
    for ele in pool:
        if count==36:
            break
        count=count+1
        attention_hour_list.append(str(ele))
    start_index=attention_hour_list.index(start_time)
    if start_time_p not in allowed_attention_list:
        attention_hour_list=[]
    else:
        num=allowed_attention_dict[start_time_p]
        end_index=start_index+num    

        attention_hour_list=attention_hour_list[start_index:end_index+1]


    return attention_hour_list

In [66]:
def get_attention_hour_list(origin_attention_df,org,start_time_p): 
    #get destination attention hour to check list which is 12 hr from origin attention obtained
    origin_attention_df_1=origin_attention_df[origin_attention_df.COD==org]
    #print("dest_atten",origin_attention_df_1)
    origin_attention_df_1=origin_attention_df_1.sort_values(by='Attention_Start')
    attention_hour_list=[]
    from itertools import cycle
    pool = cycle(origin_attention_df_1['Attention_Start'].values)
    #print(origin_attention_df_1['Attention_Start'].values)
    #print("pool",pool)
    start_time=start_time_p
    #print("inside get_attnhourlist",start_time)
    count=0
    for ele in pool:
        #print(ele)
        if count==36:
            break
        count=count+1
        attention_hour_list.append(str(ele))
    #print("attention_hour_list",attention_hour_list)
    start_index=attention_hour_list.index(start_time)
    end_index=start_index+12    

    attention_hour_list=attention_hour_list[start_index:end_index+1]

    return attention_hour_list

In [67]:
# Max_reach_time_at_destiny_p=str(current_day_fleet_siders.loc[0,'Max_reach_time_at_destiny'].time())[0:5]+":"+"00"
# fleet_start_date_time = current_day_fleet_siders.loc[0,'Fleet_start_date_time']
# fleet_start_time=str(fleet_start_date_time.hour).zfill(2)+":00:00"

In [68]:
night_time_dict={'00:00:00':23,'01:00:00':22,'02:00:00':21,'03:00:00':20,'04:00:00':19,'05:00:00':18,'06:00:00':17,'07:00:00':16,'08:00:00':15,'09:00:00':14,'10:00:00':13,
                '11:00:00':12,'12:00:00':11,'13:00:00':10,'14:00:00':9,'15:00:00':8,'16:00:00':7,'17:00:00':6,'18:00:00':5,'19:00:00':4,'20:00:00':3,'21:00:00':2,'22:00:00':1,'23:00:00':0}
night_time_attention_allowed=[]
for ele in night_time_dict.keys():
    night_time_attention_allowed.append(ele)
#night_time_attention_allowed

In [69]:
attention_list_allowed=night_time_attention_allowed
attention_dict_allowed =night_time_dict

In [70]:
def check_origin_attention_v1(origin_attention_df,origin,fleet_start_time_p,Max_reach_time_at_destiny_p,day_of_week,attention_list_allowed,attention_dict_allowed,origin_attention_df_comm):
	#check origin attention and return when can we start the sider 
	
    origin_attention_df_temp=origin_attention_df[["COD","Attention_Start","Attention_End",day_of_week+"_attention"]]
    origin_attention_df_temp=origin_attention_df_temp[origin_attention_df_temp.COD==origin]
    origin_attention_df_temp['Attention_Start']=origin_attention_df_temp['Attention_Start'].astype(str)
    origin_attention_df_comm['Attention_Start']=origin_attention_df_comm['Attention_Start'].astype(str)
    fleet_start_time=fleet_start_time_p
    Max_reach_time_at_destiny=Max_reach_time_at_destiny_p
    truck_sent =False
    attention_start_obtained="NA"
#     from itertools import cycle
#     pool = cycle(origin_attention_df_temp['Attention_Start'].values).tolist()
#     start_index=origin_attention_df_temp.Attention_Start.values.tolist().index(fleet_start_time_p)
#     end_index=start_index+iteration_checking
    #print(origin,fleet_start_time)
    attention_hour_list=get_attention_hour_list_v1(origin_attention_df,origin,fleet_start_time,attention_list_allowed,attention_dict_allowed)
    #print("origin attention hour list",attention_hour_list)
    if len(attention_hour_list)>0:
        for attention_start  in attention_hour_list:
            if attention_start==Max_reach_time_at_destiny:
                #print("max time reached trip can not be planned")
                break
            #print("currently checking for",attention_start)
            #attention_available=origin_attention_df_temp[(origin_attention_df_temp['Attention_Start']==attention_start)][day_of_week+"_attention"].values[0]
            attention_available=origin_attention_df_comm[(origin_attention_df_comm['Attention_Start']==attention_start)][day_of_week+"_attention"].values[0]
            #print("Attention available ",attention_available)
            if attention_available>0:
                attention_start_obtained=attention_start
                truck_sent=True
                break
#     print("attention_start_obtained",attention_start_obtained)
#     print("can we send truck ",truck_sent)
    return attention_start_obtained,truck_sent


In [71]:
# for j in range(current_day_fleet_siders.reset_index(drop=True).shape[0]):
#     Max_reach_time_at_destiny_p=str(current_day_fleet_siders.loc[j,'Max_reach_time_at_destiny'].time())[0:5]+":"+"00"
#     fleet_start_date_time = current_day_fleet_siders.loc[j,'Fleet_start_date_time']
#     fleet_start_time=str(fleet_start_date_time.hour).zfill(2)+":00:00"
#     check_origin_attention_v1(origin_attention_df,origin,fleet_start_time,Max_reach_time_at_destiny_p,day_of_week,attention_list_allowed,attention_dict_allowed)

In [72]:
# j=0
# Max_reach_date_time_at_destiny=current_day_fleet_siders.loc[j,'Max_reach_time_at_destiny']
# fleet_start_date_time = current_day_fleet_siders.loc[j,'Fleet_start_date_time']
# fleet_start_time=str(fleet_start_date_time.hour).zfill(2)+":00:00"
# attention_start_obtained,truck_sent=check_origin_attention_v1(origin_attention_df,origin,fleet_start_time,Max_reach_time_at_destiny_p,day_of_week,attention_list_allowed,attention_dict_allowed)

In [73]:
# distance_and_time['Destiny_Code'].unique()

In [74]:
# current_day_fleet_siders.shape

### Destination Attention check

In [75]:
# Max_reach_date_time_at_destiny_p=Max_reach_date_time_at_destiny
# attention_start_time_obtained=attention_start_obtained

In [76]:
def check_destination_attention_v1(destiny_attention_df,distance_and_time,origin,destiny,attention_start_time_obtained,Max_reach_time_at_destiny_p,fecha_str,day_of_week):
#check destination attention hour
#convert truck_start_time_obtained to date time 
    import datetime
    from datetime import timedelta
    fecha_next_str=fecha_str
    truck_start_date_time_obtained=pd.to_datetime(fecha_next_str+" "+attention_start_time_obtained, format='%d-%m-%Y %H:%M:%S')
    Max_reach_date_time_at_destiny_p=Max_reach_time_at_destiny_p
    #print("inside check_destination_attention_v1")
    #print("max reach date time to reach at desyiny ",Max_reach_date_time_at_destiny_p)
    #     if plan=='10:30 PM':
    # #         if (attention_start_time_obtained=="22:00:00" or attention_start_time_obtained=="23:00:00"):
    #         truck_start_date_time_obtained=pd.to_datetime(fecha_str+attention_start_time_obtained, format='%Y-%m-%d%H:%M:%S')
    #     else:
    #         truck_start_date_time_obtained=pd.to_datetime(fecha_next_str+attention_start_time_obtained, format='%Y-%m-%d%H:%M:%S')
    if "-I" in destiny:
        destiny1 =destiny[:-2]
    else:
        destiny1 =destiny
    wait_hour=0    
    truck_reach_dest_date_time_obtained='NA'
    truck_reach_back_date_time_obtained='NA'
    #filter destiny attention hour table     
    destiny_attention_df_temp=destiny_attention_df[["COD","Attention_Start","Attention_End",day_of_week+"_attention"]]
    destiny_attention_df_temp=destiny_attention_df_temp[destiny_attention_df_temp.COD==destiny1]
    destiny_attention_df_temp['Attention_Start']=destiny_attention_df_temp['Attention_Start'].astype(str)

    #filter distance and time and find time to reach destiny 
    transfer_info =  distance_and_time[(distance_and_time['Origin_Code'] == origin) & (distance_and_time['Destiny_Code'] == destiny)]
    #print(transfer_info)
    one_way_time_in_min = int(transfer_info['time_one_way_min'].values[0])
    two_way_time_in_min = int(transfer_info['time_two_way_min'].values[0])*2
    loading_time_origin = int(transfer_info['Loading_time_origin_min'].values[0])
    unloading_time_at_dest = int(transfer_info['unloading_loading_time_in_dest_min'].values[0])
    break_time_min_dest =int(transfer_info['break_time_min'].values[0])
    reach_at_destination_date_time=truck_start_date_time_obtained +datetime.timedelta(minutes=loading_time_origin)+datetime.timedelta(minutes=one_way_time_in_min)
    print("reach_at_destination_date_time",reach_at_destination_date_time)
    print("Max_reach_date_time_at_destiny_p",Max_reach_date_time_at_destiny_p)
    #print(reach_at_destination_date_time>Max_reach_date_time_at_destiny_p)
    #print("will reach at destiny",reach_at_destination_date_time)
    reach_at_destination_time=reach_at_destination_date_time.time()
    reach_at_destination_time_ceil_down=reach_at_destination_time.hour
    reach_at_destination_time_ceil_down_time_format=datetime.time(reach_at_destination_time_ceil_down,0,0)
    destination_atention_hour_to_check=str(reach_at_destination_time_ceil_down_time_format)
    #print("destination attention hour  to be checked ",destination_atention_hour_to_check)
    #lets check destination attention hour 

    truck_sent =False
    attention_end_obtained="NA"
    #send_external_truck=False 
    duration_of_trip='NA'
    #Break time = 0

    #STEp0: DIrectly find destination attention hour 
    
    attention_available=destiny_attention_df_temp[(destiny_attention_df_temp['Attention_Start']==destination_atention_hour_to_check)][day_of_week+"_attention"].values[0]

    #Max_reach_date_time_at_destiny_p = max time at which trip can start from origin
    #STEP 1:
    if (reach_at_destination_date_time>Max_reach_date_time_at_destiny_p):
        #print("inside if 1")
        attention_end_obtained=destination_atention_hour_to_check
        truck_sent=True
        wait_hour=0
        truck_reach_dest_date_time_obtained=reach_at_destination_date_time+datetime.timedelta(hours=wait_hour)
        truck_reach_back_date_time_obtained=truck_reach_dest_date_time_obtained+\
        datetime.timedelta(minutes=one_way_time_in_min)+datetime.timedelta(minutes=unloading_time_at_dest)
        duration_of_trip=(truck_reach_back_date_time_obtained-truck_start_date_time_obtained).seconds/3600

    else :
        #print("inside else 1")
        if attention_available>0:
            #print("inside if 2")
            attention_end_obtained=destination_atention_hour_to_check
            truck_sent=True
            #print("truck can be send in own truck ")
        else :
            #print("inside else 2")
            #print("lets try to find attention hour after waiting ")
            attention_hour_list=get_attention_hour_list(destiny_attention_df,destiny1,destination_atention_hour_to_check)
            #print(attention_hour_list)
            #wait_hour_final=0
            #print(attention_hour_list)
            for attention_dest  in attention_hour_list:
                #### we have to send the truck right??
                if attention_dest==Max_reach_time_at_destiny_p:
                    #print("max time reached to reach at destiny  trip can not be planned")
                    break
                #print("currently checking for",attention_dest)
                attention_available=destiny_attention_df_temp[(destiny_attention_df_temp['Attention_Start']==attention_dest)][day_of_week+"_attention"].values[0]
                #print("Attention available ",attention_available)
                if attention_available>0:
                    attention_end_obtained=attention_dest
                    truck_sent=True
                    break
                wait_hour=wait_hour+1
        #wait_hour=wait_hour_final
        if truck_sent==True :
            truck_reach_dest_date_time_obtained=reach_at_destination_date_time+datetime.timedelta(hours=wait_hour)
            ##### Include Honduras remolque restrictions here
            #### Add the individual restrictions here
            truck_reach_back_date_time_obtained=truck_reach_dest_date_time_obtained+\
            datetime.timedelta(minutes=one_way_time_in_min)+datetime.timedelta(minutes=unloading_time_at_dest)+\
            datetime.timedelta(minutes=break_time_min_dest)
            duration_of_trip=(truck_reach_back_date_time_obtained - truck_start_date_time_obtained)/pd.Timedelta('1 hour')

    #     print("destination attention obtained",attention_end_obtained)
    #     print("can we send external truck",send_external_truck)
    #     print("can we send own truck",truck_sent)
    #     print("waiting time ",wait_hour)
    #print(truck_reach_back_date_time_obtained,truck_start_date_time_obtained)
    #    diff_time=
    #     print("total duration of trip ",(diff_time).seconds//3600)
    #     print("total duration of trip2 ",(diff_time.seconds//60)%60)
    #    print("total duration of trip2 ",(diff_time.seconds/3600))

    return truck_start_date_time_obtained,truck_reach_dest_date_time_obtained,truck_reach_back_date_time_obtained,attention_end_obtained,truck_sent,wait_hour,one_way_time_in_min,two_way_time_in_min,unloading_time_at_dest,duration_of_trip


In [77]:
# truck_start_date_time_obtained,truck_reach_dest_date_time_obtained,truck_reach_back_date_time_obtained,\
#             attention_end_obtained,truck_sent_dest,wait_hour,one_way_time_in_min,\
#             two_way_time_in_min,unloading_time_at_dest,duration_of_trip_hr=check_destination_attention_v1(destiny_attention_df,distance_and_time,origin,destiny,attention_start_time_obtained,Max_reach_date_time_at_destiny_p,fecha_str,day_of_week)

In [78]:
#Dist_plan1.columns

### Filler SKU

In [79]:
def check_update_filler_sku_V2(max_pallets,max_weight,result_df,filler_added,Filler_sku,filler_weight):
    #find how many pallets of filler SKU required 
    #but do not add filler SKU if total weight is crossing 40 ton 
    final_df_azua=result_df   
    req_cols=[x for x in final_df_azua.columns  if x.startswith("_")]
    final_df_required=final_df_azua[req_cols]
    sum_pallets=float(final_df_required.sum(axis=1))
    sum_total_weight=float(final_df_azua['Total_weight_in_kg'])
    filler_pallets_added=0
    filler_added=False
    #print("###initial ",sum_pallets,filler_pallets_added)
    while sum_pallets<max_pallets and sum_total_weight<max_weight:
        filler_pallets_added=filler_pallets_added+1
        sum_total_weight=sum_total_weight+filler_weight
        sum_pallets=sum_pallets+1
        ##print("###inside while ",sum_pallets,filler_pallets_added)
        filler_added=True
    pallets_required=filler_pallets_added
    #print("###",sum_pallets,pallets_required)
    final_df_azua["_"+str(Filler_sku)]=final_df_azua["_"+str(Filler_sku)]+pallets_required
    final_df_azua['Total_weight_in_kg']=sum_total_weight
    final_df_azua['total_pallets_sent']=sum_pallets
    return pallets_required,filler_added

In [80]:
# def update_priority(result_df_row,df_priority_CND):
#     #df_priority_CND['codigo']=df_priority_CND['codigo'].map('-{}'.format)
#     #update priority
#     final_df_x=result_df_row
#     temp_dest=final_df_x['Destination'].values[0]
#     temp_org=final_df_x['origin'].values[0]
#     final_df_x=final_df_x.loc[:, (final_df_x != 0).any(axis=0)]
#     req_col=[x for x in final_df_x.columns  if x.startswith("_")]
#     final_df_x=final_df_x[req_col]
#     dict_temp={}
#     for k in final_df_x.columns:
#         dict_temp[int(k.split("_")[1])]=final_df_x[k].values[0]
#     df_priority_CND_required=df_priority_CND[(df_priority_CND.Destination==temp_dest)&((df_priority_CND.Origin==temp_org))]
#     #print(df_priority_CND_required)
#    # df_priority_CND_required['codigo']=df_priority_CND_required['codigo'].map('-{}'.format)
#     for sku, pallet_sider in dict_temp.items():
#         #print(sku, pallet_sider)
#         if sku in df_priority_CND_required['codigo'].to_list():
#             #print("sku updated ",sku)
#             pallet_to_be_updated=df_priority_CND_required[df_priority_CND_required.codigo==sku]['Total_pallets'].values[0]
#             #print("pallet_to_be_updated",pallet_to_be_updated)
#             #print("pallet_sider",pallet_sider)
#             #print(temp_dest)
            
#             if pallet_to_be_updated<pallet_sider:
#                 pallet_to_be_updated=0
#             else:
#                 pallet_to_be_updated=pallet_to_be_updated-pallet_sider
#             #print("pallet_to_be_updated modified",pallet_to_be_updated)
#             df_priority_CND.loc[(df_priority_CND.Destination==temp_dest) & (df_priority_CND.Origin==temp_org)&(df_priority_CND.codigo==sku),'Total_pallets']=pallet_to_be_updated
#     return df_priority_CND


def update_priority(result_df_row,Dist_plan,origin_list):
    final_df_x=result_df
    df_priority_CND = Dist_plan
    temp_dest=final_df_x['Destination'].values[0]
    final_df_x=final_df_x.loc[:, (final_df_x != 0).any(axis=0)]
    req_col=[x for x in final_df_x.columns  if x.startswith("_")]
    final_df_x=final_df_x[req_col]
    dict_temp={}
    for k in final_df_x.columns:
        dict_temp[int(k.split("_")[1])]=final_df_x[k].values[0]
    for i in range(len(origin_list)):
        temp_org=origin_list[i]
        df_priority_CND_required=df_priority_CND[(df_priority_CND.Destination==temp_dest)&((df_priority_CND.Origin==temp_org))]
        #print(df_priority_CND_required)
        # df_priority_CND_required['codigo']=df_priority_CND_required['codigo'].map('-{}'.format)
        for sku, pallet_sider in dict_temp.items():
            #print(sku, pallet_sider)
            if sku in df_priority_CND_required['codigo'].to_list():
                #print("sku updated ",sku)
                pallet_to_be_updated=df_priority_CND_required[df_priority_CND_required.codigo==sku]['Total_pallets'].values[0]
                #print("pallet_to_be_updated",pallet_to_be_updated)
                #print("pallet_sider",pallet_sider)
                #print(temp_dest)

                if pallet_to_be_updated<pallet_sider:
                    pallet_to_be_updated=0
                else:
                    pallet_to_be_updated=pallet_to_be_updated-pallet_sider
                    dict_temp[sku]=pallet_to_be_updated
                #print("pallet_to_be_updated modified",pallet_to_be_updated)
                df_priority_CND.loc[(df_priority_CND.Destination==temp_dest) & (df_priority_CND.Origin==temp_org)&(df_priority_CND.codigo==sku),'Total_pallets']=pallet_to_be_updated
    return df_priority_CND

In [81]:
# def update_origin_attention(origin_attention_df,origin,attention_start_hour_obtained,Day_of_week):
# 	#update origin attention 
#     attention=origin_attention_df.loc[((origin_attention_df.CDD==origin) &(origin_attention_df.Attention_Start.astype(str)==attention_start_hour_obtained))][Day_of_week+"_attention"]
#     attention=attention.values[0]

#     if attention>0:
#         origin_attention_df.loc[((origin_attention_df.CDD==origin) &(origin_attention_df.Attention_Start.astype(str)==attention_start_hour_obtained)),[Day_of_week+"_attention"]]=attention-1

# def update_destination_attention(destiny_attention_df,destiny,attention_end_obtained,Day_of_week):
# 	#update destination attention 
#     attention=destiny_attention_df.loc[((destiny_attention_df.CDD==destiny) &(destiny_attention_df.Attention_Start.astype(str)==attention_end_obtained))][Day_of_week+"_attention"]
#     attention=attention.values[0]
#     print(attention)

#     if attention>0:
#         destiny_attention_df.loc[((destiny_attention_df.CDD==destiny) &(destiny_attention_df.Attention_Start.astype(str)==attention_end_obtained)),[Day_of_week+"_attention"]]=attention-1


In [82]:
def rearrange_sider(current_day_fleet_siders_CND,ficha):
    #move the sider which is allocated to buttom 
    if ficha != 'NA':
        row=current_day_fleet_siders_CND.loc[current_day_fleet_siders_CND.Ficha==ficha]
        current_day_fleet_siders_CND=current_day_fleet_siders_CND[current_day_fleet_siders_CND.Ficha!=ficha]
        current_day_fleet_siders_CND=pd.concat([current_day_fleet_siders_CND,row])
        current_day_fleet_siders_CND=current_day_fleet_siders_CND.reset_index(drop=True)
    return current_day_fleet_siders_CND

In [83]:
### why arent we returning anything

def update_origin_attention(origin_attention_df,origin,attention_start_hour_obtained,Day_of_week):
	#update origin attention 
    attention=origin_attention_df.loc[((origin_attention_df.COD==origin) &(origin_attention_df.Attention_Start.astype(str)==attention_start_hour_obtained))][Day_of_week+"_attention"]
    attention=attention.values[0]

    if attention>0:
        origin_attention_df.loc[((origin_attention_df.COD==origin) &(origin_attention_df.Attention_Start.astype(str)==attention_start_hour_obtained)),[Day_of_week+"_attention"]]=attention-1

def update_destination_attention(destiny_attention_df,destiny,attention_end_obtained,Day_of_week):
	#update destination attention 
    attention=destiny_attention_df.loc[((destiny_attention_df.COD==destiny) &(destiny_attention_df.Attention_Start.astype(str)==attention_end_obtained))][Day_of_week+"_attention"]
    attention=attention.values[0]
    print(attention)

    if attention>0:
        destiny_attention_df.loc[((destiny_attention_df.COD==destiny) &(destiny_attention_df.Attention_Start.astype(str)==attention_end_obtained)),[Day_of_week+"_attention"]]=attention-1


In [84]:
def create_codigo_values(codigo_list,  total_pallets_list, weight_per_pallet_list):
    codigo_values = []
    #print(codigo_list)
    for i in range(len(codigo_list)):
        codigo_info = {
            "codigo": codigo_list[i],
            "Total_pallets": total_pallets_list[i],
            "kg_weight_per_pallet": weight_per_pallet_list[i]
        }
        codigo_values.append(codigo_info)
    return codigo_values

In [85]:
# codigo_values = create_codigo_values(vm['codigo'],vm[dest],vm['kg_weight_per_pallet'])
# x = {codigo["codigo"]: LpVariable(f"{codigo['codigo']}", lowBound=0, cat="Integer") for codigo in codigo_values}

In [86]:
def generate_recommendation_v2(vm,dest,max_pallets,max_weight):
    print("max_weight",max_weight)
    import numpy as np 
    df = pd.DataFrame()
    i=0
    dict1 = {}
    vm['kg_weight_per_pallet']=vm['kg_weight_per_pallet'].astype(float)
    while (vm[dest].sum()>0):
        print("Start of while")
        if vm[dest].sum()>=max_pallets:
            print(i,vm[dest].sum())
 
 
            codigo_values = create_codigo_values(vm['codigo'],vm[dest],vm['kg_weight_per_pallet'])
            #print(codigo_values)
 
            from pulp import LpVariable, LpProblem, LpMinimize, lpSum, LpStatus ,LpMaximize,PULP_CBC_CMD
            rnd = np.random
            rnd.seed(1)
            seed = 1 
            problem = LpProblem("PalletShippingProblem", LpMaximize)
            max_pallets = max_pallets
            max_weight = max_weight
 
            x = {codigo["codigo"]: LpVariable(f"{codigo['codigo']}", lowBound=0, cat="Integer") for codigo in codigo_values}
 
            problem += lpSum(codigo["kg_weight_per_pallet"] * x[codigo["codigo"]] for codigo in codigo_values)
            #problem += lpSum(codigo["kg_weight_per_pallet"] *24* x[codigo["codigo"]] for codigo in codigo_values)
            # # # Set the constraints
            problem += lpSum(x.values()) <= max_pallets
            #problem += lpSum(codigo["kg_weight_per_pallet"] * x[codigo["codigo"]] for codigo in codigo_values) >= min_weight
            problem += lpSum(codigo["kg_weight_per_pallet"] * x[codigo["codigo"]] for codigo in codigo_values) <= max_weight
            #problem += lpSum(codigo["kg_weight_per_pallet"] *24* x[codigo["codigo"]] for codigo in codigo_values) <= max_weight
 
            for codigo in codigo_values:
                problem +=x[codigo["codigo"]]<=codigo['Total_pallets']
 
            #problem.solve(PULP_CBC_CMD(msg=0))
            #print("prob solve started ")
            #print(problem)
            #problem.solve()
            problem.solve(PULP_CBC_CMD(msg=1, maxSeconds=0.3))
            #print("problem solve ended ")
            import numpy as np
            #print(problem)
            for var in problem.variables():
                dict1['Truck']=i
                dict1[var.name]=var.value()
                vm[dest] = np.where(vm["codigo"] == "-"+var.name.split("_")[1], vm[dest]-var.value(), vm[dest])
            dict1["Total_weight_in_kg"]=problem.objective.value()
 
            i=i+1
 
            df_temp = pd.DataFrame(dict1,index=[i])
 
            df_temp['Destination'] = dest
 
            df=df.append(df_temp)
        else:
            print("inside else ")
            print(i,vm[dest].sum())
            dict2={}
            initial_weight=0
            dict2['Truck']=i
            total_kg=0
            for m in range(vm.reset_index(drop=True).shape[0]):
                #print("*****current codigo",vm.loc[m,'codigo'])
                SKU_code= "_"+(vm.loc[m,'codigo']).split("-")[1]
                pallets= vm.loc[m,dest]
                weight_per_pallet=vm.loc[m,'kg_weight_per_pallet']
#                 print("i is", i)
#                 print("before")
#                 print(dict1)
#                 print("initial_pallet,initial_weight,pallets",initial_pallet,initial_weight,pallets)
 
#                 print(pallets)
                initial_pallet=0
                while initial_pallet<pallets:
                    initial_pallet=initial_pallet+1
                    initial_weight=initial_weight+weight_per_pallet
#                     print("inside while")
#                     print("initial_pallet,initial_weight,pallets",initial_pallet,initial_weight,pallets)
                    if initial_weight>max_weight :
                        #print("going to btreak now ")
                        initial_weight=initial_weight-weight_per_pallet
                        initial_pallet=initial_pallet-1
                        break
                dict2[SKU_code]=initial_pallet
                vm[dest] = np.where(vm["codigo"] == "-"+SKU_code.split("_")[1], vm[dest]-initial_pallet, vm[dest])
#                     else:
#                         dict2[SKU_code]=initial_pallet
#                         vm[dest] = np.where(vm["codigo"] == "-"+SKU_code.split("_")[1], vm[dest]-initial_pallet, vm[dest])
                        #pallets=pallets-1
#                 print("after")
#                 print("initial_pallet,initial_weight,pallets",initial_pallet,initial_weight,pallets)
 
#             for j in range(vm.shape[0]):
#                 sku="_"+(vm.loc[j,'codigo']).split("-")[1]
#                 pallet=vm.loc[j,dest]
#                 sku_kg=pallet*vm.loc[j,'kg_weight_per_pallet']
#                 total_kg=total_kg+sku_kg
#                 dict2[sku]=pallet
#                 vm[dest] = np.where(vm["codigo"] == "-"+sku.split("_")[1], vm[dest]-pallet, vm[dest])
 
                
            dict2["Total_weight_in_kg"]=initial_weight
            df_temp = pd.DataFrame(dict2,index=[i])
            df_temp['Destination'] = dest
            df=df.append(df_temp)
 
            
 
    return df

In [87]:
def generate_recommendation_v3(vm,dest,max_pallets,max_weight):
    #vm=vm[vm[dest]!=0].reset_index(drop=True)
    vm['kg_weight_per_pallet']=vm['kg_weight_per_pallet'].astype(float)
    vm['weight_fill']=vm['kg_weight_per_pallet']*vm[dest]
    dict2 ={}
    initial_pallet=0
    initial_weight=0
    for q in range(vm.reset_index(drop=True).shape[0]):
        SKU_code= "_"+(vm.loc[q,'codigo']).split("-")[1]
        pallets= vm.loc[q,dest]
        weight_fill=vm.loc[q,'weight_fill']
        weight_per_pallet=vm.loc[q,'kg_weight_per_pallet']
        if (pallets+initial_pallet)<=max_pallets and (weight_fill+initial_weight)<=max_weight:
            dict2[SKU_code]=pallets
            initial_weight=initial_weight+weight_fill
            initial_pallet=initial_pallet+pallets
        else:
            break
    dict2["Total_weight_in_kg"]=initial_weight
    dict2["total_pallets_sent"]=initial_pallet
    df_temp = pd.DataFrame(dict2,index=[i])
    df_temp['Destination'] = dest
    return df_temp

In [88]:
# vm = OrgPrCBefMelt1['Honduras'][['codigo','Origin','AT','kg_weight_per_pallet']].copy()
# vm['codigo']=vm['codigo'].map('-{}'.format)
# #             print(vm.shape)
# vm=vm.reset_index(drop=True)
# #vm

In [89]:
# df1=generate_recommendation_v2(vm,'AT',28,25000)
# df1=df1.head(n=1)
# df1

In [90]:
# df1

In [91]:
# req_col=[x for x in df1.columns  if x.startswith("_")]

# x1=df1[req_col]

In [92]:
# x1.sum(axis=1).values[0]

In [93]:
# Dist_plan=Dist_plan_cust2[Dist_plan_cust2['Destination']=='AT'].reset_index(drop=True)

In [94]:
# final_df_x=df1
# df_priority_CND = Dist_plan
# temp_dest=final_df_x['Destination'].values[0]
# final_df_x=final_df_x.loc[:, (final_df_x != 0).any(axis=0)]
# req_col=[x for x in final_df_x.columns  if x.startswith("_")]
# final_df_x=final_df_x[req_col]
# dict_temp={}
# for k in final_df_x.columns:
#     dict_temp[int(k.split("_")[1])]=final_df_x[k].values[0]
# dict_org_sku={}
# list1=[]
# origin_list=list(Dist_plan['Origin'].unique())
# for i in range(len(origin_list)):
#     temp_org=origin_list[i]
#     df_priority_CND_required=df_priority_CND[(df_priority_CND.Destination==temp_dest)&((df_priority_CND.Origin==temp_org))]
#     #print(df_priority_CND_required)
#     # df_priority_CND_required['codigo']=df_priority_CND_required['codigo'].map('-{}'.format)

#     for sku, pallet_sider in dict_temp.items():
#         #print(sku, pallet_sider)
#         if sku in df_priority_CND_required['codigo'].to_list():
#             #print("sku updated ",sku)
#             pallet_to_be_updated=df_priority_CND_required[df_priority_CND_required.codigo==sku]['Total_pallets'].values[0]
#             #print("pallet_to_be_updated",pallet_to_be_updated)
#             #print("pallet_sider",pallet_sider)
#             #print(temp_dest)
#             if pallet_to_be_updated<pallet_sider:
#                 pallet_to_be_updated=0
#             else:
#                 pallet_to_be_updated=pallet_to_be_updated-pallet_sider
#                 dict_temp[sku]=pallet_to_be_updated
#                 skupalletcomb= str(sku)+"-"+str(pallet_sider)
#                 print(skupalletcomb)
#                 list1.append(skupalletcomb)
#             #print("pallet_to_be_updated modified",pallet_to_be_updated)
#             df_priority_CND.loc[(df_priority_CND.Destination==temp_dest) & (df_priority_CND.Origin==temp_org)&(df_priority_CND.codigo==sku),'Total_pallets']=pallet_to_be_updated
#             dict_org_sku[temp_org]= list1   

In [95]:
# origin_list=list(Dist_plan['Origin'].unique())
# df_new=pd.DataFrame()
# for org in range(len(origin_list)):
#     Dist_cmplt=Dist_plan_cust2[(Dist_plan_cust2['Origin']==origin_list[org])&(Dist_plan_cust2['Destination']==temp_dest)].merge(df_priority_CND[(df_priority_CND['Origin']==origin_list[org])&(df_priority_CND['Destination']==temp_dest)][['codigo','Total_pallets']], on='codigo',how='left')
#     Dist_cmplt['Pallet_sent_org']=Dist_cmplt['Total_pallets_x']-Dist_cmplt['Total_pallets_y']
#     Dist_cmplt['codigo']=Dist_cmplt['codigo'].map('_{}'.format)
#     Dist_cmplt=Dist_cmplt.pivot(index='Origin', columns='codigo', values='Pallet_sent_org').reset_index().rename_axis(None, axis=1)
#     print(Dist_cmplt)
#     Dist_cmplt['Destination']=df1['Destination'].values[0]
#     Dist_cmplt['Total_weight_in_kg']=df1['Total_weight_in_kg'].values[0]
#     df_new=df_new.append(Dist_cmplt).fillna(0)
# extracols=[x for x in df1.columns  if x not in df_new.columns]
# sku_cols=[x for x in df1.columns  if x.startswith("_")]
# for newcol in extracols:
#     df_new[newcol]=0
# df_new1=df_new[sku_cols]
# df_new2=df_new[['Origin','Destination','Total_weight_in_kg']]
# result_df=pd.concat([df_new1, df_new2], axis=1)

In [96]:
# result_df

In [97]:
# x=current_day_fleet_siders.loc[0,'Fleet_start_date_time_org']

## Planning

In [98]:
#req_org_list=Dist_plan1['Origin'].unique()
req_org_list=Dist_plan1['org-priority'].unique()
req_dest_list=Dist_plan1['Destination'].unique()

#### Leave out DH14 because Roatan wil be in MVP2
req_dest_list =[x for x in req_dest_list if x != 'DH14']

In [99]:
Propia=list(current_day_fleet_siders[current_day_fleet_siders['Tipo de Vehículo']=='Propia']['Ficha'].unique())
Hibrid =list(current_day_fleet_siders[current_day_fleet_siders['Tipo de Vehículo']=='Hibrida']['Ficha'].unique())
Terceros=list(current_day_fleet_siders[current_day_fleet_siders['Tipo de Vehículo']=='Terceros']['Ficha'].unique())

propia_dest= ['DH01','DH03','DH15','DH11','DH10']
hibrid_dest= ['DH04','DH02','DH05','DH06','DH07','DH08','DH09','DH12','DH13']
Dollie =['DH00']

In [100]:
#dist_plan=dist_plan.reset_index(drop=True)
current_day_fleet_siders=current_day_fleet_siders.reset_index(drop=True)
pallets_max=0
weight_max=0
weight_max_kg=0
final_df=pd.DataFrame()
#origin='CND'
Day_of_week=day_of_week
last_assigned_ficha='NA'
iteration=0
#Dist_plan1['codigo']=Dist_plan1['codigo'].map('-{}'.format)
Filler_sku=Dist_plan_cust1[(Dist_plan_cust1['Prioridad']==1)&(Dist_plan_cust1['Total_pallets']==Dist_plan_cust1[Dist_plan_cust1['Prioridad']==1]['Total_pallets'].max())].reset_index()['codigo'][0]
filler_weight=sku_description[sku_description['codigo']==Filler_sku].reset_index()['kg_weight_per_pallet'][0]
#commented by Jitendra 
Dist_plan_cust1['Weight_KG']=Dist_plan_cust1['Total_pallets']*Dist_plan_cust1['kg_weight_per_pallet']
Dist_plan_cust1=Dist_plan_cust1.sort_values(['Prioridad','Total_pallets'],ascending=[True,False]).reset_index(drop=True)
current_day_fleet_temp = current_day_fleet_siders[current_day_fleet_siders['Ficha'].isin(Hibrid)].reset_index().drop(['index'],axis=1).append(current_day_fleet_siders[(current_day_fleet_siders['Ficha_Restriction_destiny'].isin(list(Dest_dict_Cust.keys())))&(current_day_fleet_siders['Tipo de Vehículo']=='Terceros')]).reset_index(drop=True)
while Dist_plan_cust1.Total_pallets.sum()>0:
    if iteration==2:
        break 
    #restriction 
        
    for i in range(Dist_plan_cust1.reset_index(drop=True).shape[0]):
        #print(Dist_plan1.loc[i])
        origin= Dist_plan_cust1.loc[i,'Origin']
        OneOrigin=Dist_plan_cust1.loc[i,'One_Origin']
        destiny= Dist_plan_cust1.loc[i,'Destination']
        origin_list=list(Dist_plan_cust1[Dist_plan_cust1['Destination']==destiny]['Origin'].unique())
        sku= Dist_plan_cust1.loc[i,'codigo']
        pallets=Dist_plan_cust1.loc[i,'Total_pallets']
        #code added by Jitendra 
        print("Current combination",i,origin,destiny,sku,pallets)
        if pallets ==0:
            print("we have already covered this priority lets try next one ")
            continue 
        print(Dist_plan_cust1[(Dist_plan_cust1['Origin']==origin)&(Dist_plan_cust1['Destination']==destiny)&(Dist_plan_cust1['codigo']==sku)])
        #print(Dist_plan1)
        
        if pallets ==0.0:
            #print("skipping destination sku Pallets needs to sent for this destination is 0",destiny,sku,i)
            continue 


        #current_day_fleet_temp = current_day_fleet_siders[current_day_fleet_siders['Ficha'].isin(Hibrid)].reset_index().drop(['index'],axis=1)
            
        ### Storing rest fleet in another df    
        #current_day_fleet_siders_rest=current_day_fleet_siders[current_day_fleet_siders['Ficha'].isin(list(set(list(current_day_fleet_siders['Ficha'].unique())).difference(list(current_day_fleet_temp['Ficha'].unique()))))].reset_index().drop(['index'],axis=1)

        for j in range(current_day_fleet_temp.shape[0]):
            print("origin is",origin_list)
            print("j started",j)
            print("Truck Name",current_day_fleet_temp['Ficha'].loc[j])
            print("destiny is",destiny)
            print("pallets left",OrgPrCBefMelt1[OneOrigin][destiny].sum())
            #code added by Jitendra 
            if Dist_plan_cust1.loc[i,'Total_pallets']<=0:
                print("The current Org dest SKU combination satisfied lets move to other ")
                break
            restr=current_day_fleet_temp.loc[j]['Ficha_Restriction_destiny']
            Type =current_day_fleet_temp.loc[j]['Tipo de Vehículo']
            if "-I" in restr:
                restr_1 =restr[:-2]
                dest1 =restr
            else:
                restr_1=restr
                dest1 =destiny
            #origin = org[:4]
            #destiny1 =dest1
            
            ### changing 'Origin' to 'OneOrgin'
            if (Dist_plan_cust1[(Dist_plan_cust1['One_Origin']==OneOrigin)&(Dist_plan_cust1['Destination']==destiny)]['Total_pallets'].sum()<=28)& ((current_day_fleet_temp.loc[j,'Capacidad KG'])<Dist_plan_cust1[(Dist_plan_cust1['One_Origin']==OneOrigin)&(Dist_plan_cust1['Destination']==destiny)]['Weight_KG'].sum())&(Dist_plan_cust1[(Dist_plan_cust1['One_Origin']==OneOrigin)&(Dist_plan_cust1['Destination']==destiny)]['Weight_KG'].sum() < current_day_fleet_siders['Capacidad KG'].max()):
                print(current_day_fleet_temp['Ficha'].loc[j],"truck cant be used as max capacity not enough for entire load")
                print("Total weight :", Dist_plan_cust1[(Dist_plan_cust1['One_Origin']==OneOrigin)&(Dist_plan_cust1['Destination']==destiny)]['Weight_KG'].sum())
                print("Max Capacity in list of trucks :", current_day_fleet_siders['Capacidad KG'].max())
                continue
            if (restr_1!='No restriction') and (destiny != restr_1):
                print(current_day_fleet_temp['Ficha'].loc[j],"truck will not go to dest due to restriction of",restr)
                continue   
            if OrgPrCBefMelt1[OneOrigin][destiny].sum()<1:
                #print("Skipping this sider as sum of pallets found zero or less than 1",dest)
                break
            own_rented='company_owned'
            update_origin_attention_flag=False
            update_dest_attention_flag=False
            update_dist_plan=False
            update_sider=False
            filler_sku_added=False
            filler_added=False
            update_sider_start_time=False 
            can_send_in_own_sider=False 
#             can_send_in_rented_sider=False
#             print("j is ",j)
#             print(current_day_fleet_temp.shape)
            ficha=current_day_fleet_temp.loc[j,'Ficha']
            fleet_start_date_time=current_day_fleet_temp.loc[j,'Fleet_start_date_time']
            fleet_start_date_time_org=current_day_fleet_temp.loc[j,'Fleet_start_date_time_org']
            fleet_start_time=str(fleet_start_date_time.hour).zfill(2)+":00:00"
            Max_reach_time_at_destiny=str(current_day_fleet_temp.loc[j,'Max_reach_time_at_destiny'].time())[0:5]+":"+"00"
            Max_reach_date_time_at_destiny=current_day_fleet_temp.loc[j,'Max_reach_time_at_destiny']

            #print("currently_checking_for_ficha", ficha)
            if (fleet_start_date_time>Max_reach_date_time_at_destiny):
                print(ficha,"truck will not go as start time at origin is > maximum time it should start from origin")
                #print("This sider can not go  as start time at origin is > maximum time it should start from origin   ",fleet_start_date_time,Max_reach_date_time_at_destiny)
                continue
            
 ####################### Code block to accomodate same destination multiple origins
            #### Making a common attention list for all origins
            if len(origin_list)==1:
                origin_attention1=origin_attention_df[origin_attention_df['COD']==origin_list[0]].reset_index(drop=True)
            else:
                origin_attention_dict={}
                for org in range(len(origin_list)):
                    origin_attention_dict[origin_list[org]]=origin_attention_df[(origin_attention_df['COD']==origin_list[org])]
                origin_attention1 =pd.DataFrame()
                for k in origin_attention_dict.keys():
                    if origin_attention1.shape[0]==0:
                        origin_attention1=origin_attention_dict[k]
                    else:
                        origin_attention1=origin_attention1.merge(origin_attention_dict[k], on=['Attention_Start','Attention_End'],how='outer')
                        origin_attention1['Weekday_attention']=origin_attention1['Weekday_attention_x']*origin_attention1['Weekday_attention_y']
                        origin_attention1['Saturday_attention']=origin_attention1['Saturday_attention_x']*origin_attention1['Saturday_attention_y']
                        origin_attention1['Sunday_attention']=origin_attention1['Sunday_attention_x']*origin_attention1['Sunday_attention_y']
                        origin_attention1=origin_attention1[['Attention_Start','Attention_End','Weekday_attention','COD_x','CDD_x','Saturday_attention','Sunday_attention']].rename(columns={'COD_x':'COD','CDD_x':'CDD'}).reset_index(drop=True)
                        #origin_attention1=origin_attention1.merge(origin_attention_dict[k], on=['Attention_Start','Attention_End'],how='outer')[['Attention_Start','Attention_End','Weekday_attention_x','COD_x','CDD_x','Saturday_attention_x','Sunday_attention_x']].rename(columns={'Weekday_attention_x':'Weekday_attention','COD_x':'COD','CDD_x':'CDD','Saturday_attention_x':'Saturday_attention','Sunday_attention_x':'Sunday_attention'}).reset_index(drop=True)
                # origin_attention1=origin_attention1.reset_index(drop=True)
               
            attention_start_time_obtained,truck_sent_origin=check_origin_attention_v1\
            (origin_attention_df,origin_list[0],fleet_start_time,Max_reach_time_at_destiny,day_of_week,\
            attention_list_allowed,attention_dict_allowed,origin_attention1) 
  ######################## Code block end to accomodate multiple origins and same destination ################
            if truck_sent_origin==False:
                print(ficha,"truck will not start from origin as we are not getting origin attention")
                #print("This sider can not go  as we are not getting origin attention lets look for other sider  ",ficha)
                continue
            #print("can start",attention_start_time_obtained)

            #check destination attention 
            truck_start_date_time_obtained,truck_reach_dest_date_time_obtained,truck_reach_back_date_time_obtained,\
            attention_end_obtained,truck_sent_dest,wait_hour,one_way_time_in_min,\
            two_way_time_in_min,unloading_time_at_dest,duration_of_trip_hr=check_destination_attention_v1(destiny_attention_Custdf,distance_and_time_cust,origin,dest1,attention_start_time_obtained,Max_reach_date_time_at_destiny,fecha_str,day_of_week)

            if truck_sent_dest==False:
                print(ficha,"truck will not go to dest as we are not getting dest attention")
                #print("This sider can not go to destination even after waiting  ",ficha)
                continue
            print("truck_sent_dest",truck_sent_dest)
            #print("can_send_in_own_sider",can_send_in_own_sider)
            #print("can_send_in_rented_sider",can_send_in_rented_sider)
            if truck_sent_dest==True:
                print(ficha,"truck will go to dest")
                #print("company owned sider will be used ")
                can_send_in_own_sider=True
                update_origin_attention_flag=True
                update_dest_attention_flag=True
                update_sider_start_time=True

            if can_send_in_own_sider==True:
                pallets_max=current_day_fleet_temp.loc[j,'Capacidad Pallets']
                weight_max=current_day_fleet_temp.loc[j,'Capacidad Ton']
                weight_max_kg=current_day_fleet_temp.loc[j,'Capacidad KG']

            max_pallets=pallets_max
            max_weight=weight_max_kg
#             print("can_send_in_own_sider",can_send_in_own_sider)
#             print("can_send_in_rented_sider",can_send_in_rented_sider)
#             print("MAximum",max_pallets, max_weight)

            dest=destiny

            #OrgPrCBefMelt1[origin]['codigo']=OrgPrCBefMelt1[origin]['codigo'].map('-{}'.format)
            vm = OrgPrCBefMelt1[OneOrigin][['codigo',dest,'kg_weight_per_pallet']].copy()
            vm['codigo']=vm['codigo'].map('-{}'.format)
#             print(vm.shape)
            vm=vm.reset_index(drop=True)
#             print(vm.head(2))
#             from pulp import LpVariable, LpProblem, LpMinimize, lpSum, LpStatus ,LpMaximize
            df1=generate_recommendation_v2(vm,dest,max_pallets,max_weight)
            total_pallets_sent=df1[[x for x in df1.columns  if x.startswith("_")]].sum(axis=1)
            df1['total_pallets_sent']=total_pallets_sent
            #df1=df1.sort_values(by ="_"+str(sku),ascending = False).head(n=1)
            df1=df1.sort_values(['total_pallets_sent','Total_weight_in_kg'],ascending = [False,False]).head(n=1)
            result_df=df1.head(1)
#             df1=df1.sort_values(by ="_"+str(sku),ascending = False).head(n=1)
            result_string = ','.join(map(str, origin_list))
            result_df["Overall_origin"]=result_string   
            #result_df['origin']=origin
            result_df['Fleet']=ficha
            result_df['Type']=Type
#             result_df['Rest-Individual']=dest1
#             result_df['Rest-Individual'].fillna('no rest')
#             #print(result_df)
            Dist_plan_cust1_bef=Dist_plan_cust1.copy(deep=True)
            Dist_plan_cust1=update_priority(df1,Dist_plan_cust1,origin_list)
            Dist_plan_cust1=Dist_plan_cust1.reset_index().drop(['index'],axis=1)
            df_new=pd.DataFrame()
            #df_check=pd.DataFrame()
            for org in range(len(origin_list)):
                Dist_cmplt=Dist_plan_cust1_bef[(Dist_plan_cust1_bef['Origin']==origin_list[org])&(Dist_plan_cust1_bef['Destination']==destiny)].merge(Dist_plan_cust1[(Dist_plan_cust1['Origin']==origin_list[org])&(Dist_plan_cust1['Destination']==destiny)][['codigo','Total_pallets']], on='codigo',how='left')
                Dist_cmplt['Pallet_sent_org']=Dist_cmplt['Total_pallets_x']-Dist_cmplt['Total_pallets_y']
                Dist_cmplt['codigo']=Dist_cmplt['codigo'].map('_{}'.format)
                #df_check=df_check.append(Dist_cmplt)
                Dist_cmplt=Dist_cmplt.pivot(index='Origin', columns='codigo', values='Pallet_sent_org').reset_index().rename_axis(None, axis=1)
                #print(Dist_cmplt)
                Dist_cmplt['Pallets_sent']=Dist_cmplt[[pal for pal in Dist_cmplt.columns  if pal.startswith("_")]].sum(axis=1)
                Dist_cmplt['Destination']=result_df['Destination'].values[0]
                Dist_cmplt['Total_weight_in_kg']=result_df['Total_weight_in_kg'].values[0]
                Dist_cmplt['Fleet']=result_df['Fleet'].values[0]
                Dist_cmplt['Type']=result_df['Type'].values[0]
                Dist_cmplt['Overall_origin']=result_df['Overall_origin'].values[0]
                Dist_cmplt['total_pallets_sent']=result_df['total_pallets_sent'].values[0]
                df_new=df_new.append(Dist_cmplt).fillna(0)
            extracols=[x for x in result_df.columns  if x not in df_new.columns]
            sku_cols=[x for x in result_df.columns  if x.startswith("_")]
            for newcol in extracols:
                df_new[newcol]=0
            df_new1=df_new[sku_cols]
            df_new2=df_new[['Origin','Overall_origin','Fleet','Type','Destination','Total_weight_in_kg','total_pallets_sent','Pallets_sent']]
            result_df_new=pd.concat([df_new1, df_new2], axis=1)
# #             Dist_plan1=Dist_plan1.sort_values(['Prioridad'],ascending=[True])
            OrgPr_C1={}
            for m in list(Dist_plan_cust1['One_Origin'].unique()):
                #print(i)
                temp = Dist_plan_cust1[Dist_plan_cust1['One_Origin']==m]
                OrgPr_C1[m]=temp

            OrgPrCBefMelt1={}
            for n in list(OrgPr_C1.keys()):
                #print(i) 
                temp1=OrgPr_C1[n].pivot(index=['Origin','codigo'], columns='Destination', values='Total_pallets').fillna(0).reset_index().rename_axis(None, axis=1)
                OrgPrCBefMelt1[n]=temp1
                #### Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of destinities-total 15.
                diff_col=list(set(All_destiny_cust).difference(list(OrgPrCBefMelt1[n].columns)))
                for l in diff_col:
                    OrgPrCBefMelt1[n][l]=0

            # Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of SKUs.
            for p in OrgPrCBefMelt1.keys():
                OrgPrCBefMelt1[p]=OrgPrCBefMelt1[p].append(pd.DataFrame(list(set(All_sku_cust).difference(list(OrgPrCBefMelt1[p]['codigo'].unique()))), columns=['codigo'])).fillna(0).reset_index().drop(['index'],axis=1)
                OrgPrCBefMelt1[p]=OrgPrCBefMelt1[p].merge(sku_description[['codigo','kg_weight_per_pallet']])

#             total_pallets_sent=result_df[[x for x in result_df.columns  if x.startswith("_")]].sum(axis=1)

#             result_df["total_pallets_sent"]=total_pallets_sent
#             #result_df["own_rented"]=own_rented
            result_df_new["Original_sider_start_date_time"]=fleet_start_date_time_org
            result_df_new["Origin_attention_obtained_sider"]=truck_start_date_time_obtained
            result_df_new["one_way_time_in_min"]=one_way_time_in_min
            #result_df["two_way_time_in_min"]=two_way_time_in_min
            result_df_new["unloading_time_at_dest"]=unloading_time_at_dest

            result_df_new["sider_reach_dest_date_time_obtained"]=truck_reach_dest_date_time_obtained
            result_df_new["Sider_wait_time_at_destiny"]=wait_hour
            result_df_new["sider_reach_back_date_time_obtained"]=truck_reach_back_date_time_obtained
            result_df_new["total_trip_time_hr"]=duration_of_trip_hr
            

#             pallets_required_filler,filler_added= check_update_filler_sku_V2(max_pallets,max_weight,result_df,filler_added,Filler_sku,filler_weight)

#             result_df["filler_added"]=filler_added
#             result_df["filler_added_pallets"]=pallets_required_filler
#             result_df["priority"]="Priority"
#             #result_df["ficha_sider"]=ficha

            #UPDATE ORIGIN ATTENTION
    
            if update_origin_attention_flag==True:
                for org1 in range(len(origin_list)):
                    #print(origin,attention_start_time_obtained,Day_of_week)
                    update_origin_attention(origin_attention_df,origin_list[org1],attention_start_time_obtained,Day_of_week)

            #UPDATE DESTINATION ATTENTION
            if update_dest_attention_flag==True:
                update_destination_attention(destiny_attention_Custdf,destiny,attention_end_obtained,Day_of_week)

            #UPDATE SIDER START DATE TIME AS ORIGIN ATTENTION OBTAINED 
            if update_sider_start_time==True :
                current_day_fleet_temp.loc[current_day_fleet_temp.Ficha==ficha,'Fleet_start_date_time']=truck_reach_back_date_time_obtained

            final_df=pd.concat([final_df,result_df_new]).fillna(0) 
            final_df=final_df.reset_index(drop=True)
            last_assigned_ficha=ficha

#             current_day_fleet_siders=current_day_fleet_temp.append(current_day_fleet_siders_rest).reset_index().drop(['index'],axis=1)
            current_day_fleet_temp=rearrange_sider(current_day_fleet_temp,last_assigned_ficha)
#             #code added by Jitendra 
            print("$$$$$",Dist_plan_cust1.loc[i,'Total_pallets'] )
            print(i,j)
    iteration=iteration+1


Current combination 0 SH01 AT 17365 23.0
  Cod Origin Destination codigo  Prioridad  Total_boxes what is total unit  \
0  AT   SH01          AT  17365          1          805          SH01-DH16   

    Route  kg_weight_per_pallet  package_per_pallet  Total_pallets  \
0  HNPP10           1103.135744                35.0           23.0   

  org-priority One_Origin     Weight_KG  
0       SH01-1   Honduras  25372.122112  
origin is ['SH01', 'BH01']
j started 0
Truck Name HN-HC05  HN-MR78
destiny is AT
pallets left 78.0
reach_at_destination_date_time 2023-09-14 06:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
truck_sent_dest True
HN-HC05  HN-MR78 truck will go to dest
max_weight 25926.230656
Start of while
0 78.0
Start of while
1 50.0
Start of while
inside else 
2 24.0
1.0
$$$$$ 16.0
0 0
origin is ['SH01', 'BH01']
j started 1
Truck Name HN-EC32  HN-MR91
destiny is AT
pallets left 50.0
reach_at_destination_date_time 2023-09-14 06:30:00
Max_reach_date_time_at_destiny_p 2023-09-1

1.0
$$$$$ 0.0
11 0
origin is ['SH01', 'BH01']
j started 1
Truck Name HN-EC08  HN-MR56
destiny is AC
pallets left 0.0
The current Org dest SKU combination satisfied lets move to other 
Current combination 12 SH01 AC 14051 0.0
we have already covered this priority lets try next one 
Current combination 13 SH01 AH 14051 0.0
we have already covered this priority lets try next one 
Current combination 14 SH01 AH 14619 0.0
we have already covered this priority lets try next one 
Current combination 15 BH01 BP 13919 6.0
   Cod Origin Destination codigo  Prioridad  Total_boxes what is total unit  \
15  BP   BH01          BP  13919          1          216          BH01-DH16   

     Route  kg_weight_per_pallet  package_per_pallet  Total_pallets  \
15  HNBB11            689.078823                36.0            6.0   

   org-priority One_Origin    Weight_KG  
15       BH01-1   Honduras  4134.472936  
origin is ['BH01', 'SH01']
j started 0
Truck Name HN-EC12  HN-VR07
destiny is BP
pallets left 2

reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
HN-XC27  HN-VR31 truck will not go to dest as we are not getting dest attention
origin is ['SH01', 'BH01']
j started 24
Truck Name HN-NC03  HN-MR76
destiny is AQ
pallets left 27.0
reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
HN-NC03  HN-MR76 truck will not go to dest as we are not getting dest attention
origin is ['SH01', 'BH01']
j started 25
Truck Name HN-XC11  HN-VR56
destiny is AQ
pallets left 27.0
reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
HN-XC11  HN-VR56 truck will not go to dest as we are not getting dest attention
origin is ['SH01', 'BH01']
j started 26
Truck Name HN-XC17  HN-VR62
destiny is AQ
pallets left 27.0
reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
HN-XC17  HN-VR62 truck will not go to dest a

reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
HN-YC03  HN-VR26 truck will not go to dest as we are not getting dest attention
origin is ['SH01', 'BH01']
j started 55
Truck Name HN-EC20  HN-MR87
destiny is AQ
pallets left 27.0
reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
HN-EC20  HN-MR87 truck will not go to dest as we are not getting dest attention
origin is ['SH01', 'BH01']
j started 56
Truck Name HN-EC24  HN-VR40
destiny is AQ
pallets left 27.0
reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
HN-EC24  HN-VR40 truck will not go to dest as we are not getting dest attention
origin is ['SH01', 'BH01']
j started 57
Truck Name HN-TC13  HN-MR84
destiny is AQ
pallets left 27.0
reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
HN-TC13  HN-MR84 truck will not go to dest a

reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
HN-TC16  HN-MR45 truck will not go to dest as we are not getting dest attention
origin is ['SH01', 'BH01']
j started 88
Truck Name HN-TC26  HN-MR51
destiny is AQ
pallets left 27.0
reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
HN-TC26  HN-MR51 truck will not go to dest as we are not getting dest attention
origin is ['SH01', 'BH01']
j started 89
Truck Name HN-TC02  HN-MR39
destiny is AQ
pallets left 27.0
reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
HN-TC02  HN-MR39 truck will not go to dest as we are not getting dest attention
origin is ['SH01', 'BH01']
j started 90
Truck Name HN-NC01  HN-MR71
destiny is AQ
pallets left 27.0
reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
HN-NC01  HN-MR71 truck will not go to dest a

HN-TC17  HN-MR53 truck will not go to dest as we are not getting dest attention
origin is ['SH01', 'BH01']
j started 121
Truck Name HN-XC14  HN-VR50
destiny is AQ
pallets left 27.0
reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
HN-XC14  HN-VR50 truck will not go to dest as we are not getting dest attention
origin is ['SH01', 'BH01']
j started 122
Truck Name HN-UC01  HN-VR93
destiny is AQ
pallets left 27.0
reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
HN-UC01  HN-VR93 truck will not go to dest as we are not getting dest attention
origin is ['SH01', 'BH01']
j started 123
Truck Name HN-EC30  HN-VR16
destiny is AQ
pallets left 27.0
reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
HN-EC30  HN-VR16 truck will not go to dest as we are not getting dest attention
origin is ['SH01', 'BH01']
j started 124
Truck Name HN-TC01  HN-V

In [101]:
Dist_plan_cust1['Total_pallets'].sum()

0.0

In [102]:
# final_df.to_excel('Data/Output/Customer/1409/Result_Honduras_14thSep_Cust_wide_check2.xlsx')

In [1416]:
final_df['trip_id']=final_df['total_trip_time_hr'].astype(str)+"-"+final_df['Destination']+"-"+final_df['Fleet']
final_df['sl.no']=0
final_df1=pd.DataFrame()
trip=0
for i in (final_df['trip_id'].unique()):
#    print(final_df[final_df['trip_id']==i]['Destination'])
    final_temp=final_df[final_df['trip_id']==i].reset_index(drop=True)
    trip=trip+1
    final_temp.loc[:,'sl.no']=trip
    final_df1=final_df1.append(final_temp)
final_df1["only_date"] = [d.date() for d in final_df1["Original_sider_start_date_time"]]
final_df1['day']=pd.to_datetime(final_df1["only_date"], format='%Y-%m-%d').dt.day
final_df1['month']=pd.to_datetime(final_df1["only_date"], format='%Y-%m-%d').dt.month
final_df1['year']=pd.to_datetime(final_df1["only_date"], format='%Y-%m-%d').dt.year
# final_df1['sl.no']=list(np.arange(1,(final_df1.shape[0]+1)))
# final_df1['sl.no']=final_df1['sl.no'].apply(lambda x: '{0:0>5}'.format(x))
final_df1['Trip_no']="H"+final_df1['year'].astype(str).str[-2:] + final_df1['month'].astype(str)+final_df1['day'].astype(str)+final_df1['sl.no'].astype(str)
final_df1=final_df1.rename(columns={'Origin':'origin'})

In [1417]:
final_df1.to_excel('Data/Output/Customer/1409/Result_Honduras_14thSep_Cust_wide_v04.xlsx')

In [1418]:
sku_col=[col for col in final_df if col.startswith('_')]
final_df2=pd.concat([final_df1[sku_col],final_df1[['Total_weight_in_kg', 'Destination', 'origin','Overall_origin',
       'Fleet', 'Type', 'total_pallets_sent',
       'Original_sider_start_date_time', 'Origin_attention_obtained_sider',
       'one_way_time_in_min', 'unloading_time_at_dest',
       'sider_reach_dest_date_time_obtained', 'Sider_wait_time_at_destiny',
       'sider_reach_back_date_time_obtained', 'total_trip_time_hr',
       'Trip_no']]],axis=1).reset_index().drop(['index'],axis=1)

In [1419]:
final_df_long=(pd.melt(final_df2, id_vars=['Trip_no','Destination','origin','Overall_origin','Total_weight_in_kg',
       'Fleet', 'Type','total_pallets_sent','Original_sider_start_date_time', 'Origin_attention_obtained_sider',
       'one_way_time_in_min', 'unloading_time_at_dest',
       'sider_reach_dest_date_time_obtained', 'Sider_wait_time_at_destiny',
       'sider_reach_back_date_time_obtained', 'total_trip_time_hr'], var_name="SKU"))
final_df_long=final_df_long[final_df_long['value']!=0]
final_df_long=final_df_long.rename(columns={'value':'Total Pallets sent'})
final_df_long['SKU']=final_df_long['SKU'].str[1:]

In [1420]:
final_df_long=final_df_long[['Trip_no','origin','Overall_origin', 'Destination', 'Total_weight_in_kg', 'total_pallets_sent','Fleet', 'Type',
       'Original_sider_start_date_time', 'Origin_attention_obtained_sider',
       'one_way_time_in_min', 'unloading_time_at_dest',
       'sider_reach_dest_date_time_obtained', 'Sider_wait_time_at_destiny',
       'sider_reach_back_date_time_obtained', 'total_trip_time_hr','SKU', 'Total Pallets sent']].reset_index(drop=True)

In [1421]:
final_df_long.to_excel('Data/Output/Customer/1409/Result_Honduras_14thSep_Cust_long_v04.xlsx',index=False)

In [107]:
final_df_long['SKU'].str[1:]

35      13908
187     13910
198     13910
205     13910
215     13910
        ...  
7579    18410
7583    18410
7587    18410
7590    18410
7594    18410
Name: SKU, Length: 945, dtype: object